# Import libraries

In [ ]:
#Import Libraries

from eppy import modeleditor
from eppy.modeleditor import IDF
import os
import pandas as pd 
import glob
import datetime
import plotly as plty
import plotly.express as px
import calendar
import numpy as np
import re
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
from IPython.display import display, HTML

# Load energy model and set output path

In [ ]:
#Load Model

##Energy model input
idf_file=r"C:\Users\Desktop\EP.idf"

#Energyplus idd file input
iddfile =r"C:\EnergyPlusV22-1-0\Energy+.idd"

#Weather file input
epwfile=r"C:\Users\Desktop\WeatherFile\CAN_ON_TORONTO-INTL-A_6158731_CWEC.epw"
    
#Set idd file
IDF.setiddname(iddfile)

#Make copy of original idf file(optional) 
idf_original = IDF(idf_file)

#Set model object for conversion
idf1 = IDF(idf_file)

#Set Output File Name
output_filename=idf_file.rsplit('\\', 1)[-1].rsplit('.', 1)[0]
output_filename=output_filename+"_TRConverted.idf"

#Set Output Path
idf_output_folder=r"C:\Users\Desktop\Output"

print(output_filename)


# Input - Define Disruptive Event

In [ ]:
# Define Disruptive Event

Event_StartYear="2023"

# Date format is "Month/Day"
Event_StartDate="7/15"

# Time format is "hour:minute"in 24 hour scale - 0:00 to 23:00
Event_StartTime="06:00"

Event_EndYear="2023"

# Date format is "Month/Day"
Event_EndDate="7/23"

# Time format is "hour:minute"in 24 hour scale - 0:00 to 23:00
Event_EndTime="23:00"


# Compile disruptive event start and end date/time

In [ ]:
def compile_date(Event_StartYear,Event_StartDate,Event_StartTime,Event_EndYear,Event_EndDate,Event_EndTime):
######Purpose of this function is to convert user entered date time into variables##########
######Inputs of the function are user input event start and end date/time##################
######Outputs of this function are variables that will be used in downstream functions######

    Event_StartDate_Year=int(Event_StartYear)
    Event_StartDate_Month=int(Event_StartDate.split("/")[0])
    Event_StartDate_Day=int(Event_StartDate.split("/")[1])
    Event_StartDate_Hour=int(Event_StartTime.split(":")[0])
    Event_StartDate_Minute=int(Event_StartTime.split(":")[1])

    Event_EndDate_Year=int(Event_EndYear)
    Event_EndDate_Month=int(Event_EndDate.split("/")[0])
    Event_EndDate_Day=int(Event_EndDate.split("/")[1])
    Event_EndDate_Hour=int(Event_EndTime.split(":")[0])
    Event_EndDate_Minute=int(Event_EndTime.split(":")[1])

    return(Event_StartDate_Year,Event_StartDate_Month,Event_StartDate_Day,Event_StartDate_Hour,Event_StartDate_Minute,Event_EndDate_Year,Event_EndDate_Month,Event_EndDate_Day,Event_EndDate_Hour,Event_EndDate_Minute)
    
Event_StartDate_Year,Event_StartDate_Month,Event_StartDate_Day,Event_StartDate_Hour,Event_StartDate_Minute,Event_EndDate_Year,Event_EndDate_Month,Event_EndDate_Day,Event_EndDate_Hour,Event_EndDate_Minute=compile_date(Event_StartYear,Event_StartDate,Event_StartTime,Event_EndYear,Event_EndDate,Event_EndTime)

In [ ]:
#Create DateTime object for event start and end date

## If the period start and end date of simulation is not filled in, event start/end year needs to be selected based on the weekday that simulation starts, this is to allow the use of date time functions in the following steps

RunPeriod_idf=idf1.idfobjects['RunPeriod'][0]

if RunPeriod_idf.Begin_Year == "" and RunPeriod_idf.Day_of_Week_for_Start_Day == "Sunday":
    
    RunPeriod_Year=2023

elif RunPeriod_idf.Begin_Year == "" and RunPeriod_idf.Day_of_Week_for_Start_Day == "Monday":
    
    RunPeriod_Year=2018
    
elif RunPeriod_idf.Begin_Year == "" and RunPeriod_idf.Day_of_Week_for_Start_Day == "Tuesday":
    
    RunPeriod_Year=2019
    
elif RunPeriod_idf.Begin_Year == "" and RunPeriod_idf.Day_of_Week_for_Start_Day == "Wednesday":
    
    RunPeriod_Year=2014
    
elif RunPeriod_idf.Begin_Year == "" and RunPeriod_idf.Day_of_Week_for_Start_Day == "Thursday":
    
    RunPeriod_Year=2015
    
elif RunPeriod_idf.Begin_Year == "" and RunPeriod_idf.Day_of_Week_for_Start_Day == "Friday":
    
    RunPeriod_Year=2010
    
elif RunPeriod_idf.Begin_Year == "" and RunPeriod_idf.Day_of_Week_for_Start_Day == "Saturday":
    
    RunPeriod_Year=2011
    
else:
    RunPeriod_Year=int(RunPeriod_idf.Begin_Year)
    
Event_Start_Datetime = datetime.datetime(RunPeriod_Year,Event_StartDate_Month,Event_StartDate_Day,Event_StartDate_Hour,Event_StartDate_Minute)
Event_End_Datetime = datetime.datetime(RunPeriod_Year,Event_EndDate_Month,Event_EndDate_Day,Event_EndDate_Hour,Event_EndDate_Minute)

# Create analysis table to indicate whether event start and end date falls under a weekday or weekend

EventDate_Analyze_df=pd.DataFrame({'Event Date':[Event_Start_Datetime,Event_End_Datetime]})
EventDate_Analyze_df["Is_Weekday"]=EventDate_Analyze_df['Event Date'].dt.dayofweek.between(0,4)
EventDate_Analyze_df["Is_Weekend"]=EventDate_Analyze_df['Is_Weekday'] == False

EventDate_Analyze_df

In [ ]:
#The tool needs to determine whether the event start and end dates are special days such as holidays, 
#and if not, which weekday they belong to

##Function to translate special day date description in energyplus model into python date time object

def RunPeriodSpecial(Special_Date):
######Purpose of this function is to translate special day date description in energyplus model into python date time object##########
######Input of the function is a special date in energyplus idf file##################################################################
######Output of this function is a formatted python date object#######################################################################

    WeekDayList={'Monday':0, 'Tuesday':1, 'Wednesday':2, 'Thursday':3, 'Friday':4, 'Saturday':5, 'Sunday':6}
    MonthList={'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8, 'September':9, 'October':10, 'November':11, 'December':12}
    Special_Date=Special_Date
    if len(Special_Date.split())==2:
         Special_Date_Month=MonthList[str(Special_Date.split(" ")[0])]
         Special_Date_Day=int(Special_Date.split(" ")[1])
         Special_Date_formatted=datetime.date(RunPeriod_Year,Special_Date_Month,Special_Date_Day)             
    elif len(Special_Date.split())==4:
         Special_Date_NumDay=str(Special_Date.split(" ")[0])
         Special_Date_WeekDay=WeekDayList[str(Special_Date.split(" ")[1])]
         Special_Date_Month=MonthList[str(Special_Date.split(" ")[3])]
        
         if len(Special_Date_NumDay)==3:
             Special_Date_NumDay=int(Special_Date_NumDay[0])
             Special_Date_formatted=calendar.Calendar(Special_Date_WeekDay).monthdatescalendar(RunPeriod_Year, Special_Date_Month)[Special_Date_NumDay][0]
         elif Special_Date_NumDay=="Last" or Special_Date_NumDay=="last":
             Special_Date_formatted=calendar.Calendar(Special_Date_WeekDay).monthdatescalendar(RunPeriod_Year, Special_Date_Month)[-1][0]
    
    return Special_Date_formatted

## Function to list all special days in an energyplus model with formatted python datetime

def RunPeriodSpecialDays(idf1):
######Purpose of this funciton is to list all special days in an energyplus model with formatted python datetime#####################################
######Input of the funciton is the energyplus idf file(eppy object)##################################################################################
######Output of this funciton is a pandas dataframe with all special days in an energyplus model with formatted python datetime######################
    
    #Special Days
    idf_RunPeriodSpecialDays=idf1.idfobjects["RUNPERIODCONTROL:SPECIALDAYS"]
    idf_RunPeriodSpecialDays_len=len(idf_RunPeriodSpecialDays)

    ##list all Special Days - Holiday
    SpecialDay_list=[]
    StartDate_list=[]
    Duration_list=[]
    SpecialDay_Type_list=[]

    SpecialDay_list=[SpecialDay["Name"] for SpecialDay in idf_RunPeriodSpecialDays]
    StartDate_list=[SpecialDay["Start_Date"] for SpecialDay in idf_RunPeriodSpecialDays]
    Duration_list=[SpecialDay["Duration"] for SpecialDay in idf_RunPeriodSpecialDays]
    SpecialDay_Type_list=[SpecialDay["Special_Day_Type"] for SpecialDay in idf_RunPeriodSpecialDays]
    
    
    RunPeriodSpcialDay_list=pd.DataFrame({'Name':SpecialDay_list,'Start_Date':StartDate_list, 'Duration':Duration_list, 'Special_Day_Type':SpecialDay_Type_list})
    
    RunPeriodSpcialDay_list['Start_Date_Formatted']=pd.to_datetime(RunPeriodSpcialDay_list['Start_Date'].apply(RunPeriodSpecial))
    RunPeriodSpcialDay_list['Duration']=pd.to_timedelta(RunPeriodSpcialDay_list['Duration'],'d')
    RunPeriodSpcialDay_list['End_Date_Formatted']=RunPeriodSpcialDay_list['Start_Date_Formatted']+RunPeriodSpcialDay_list['Duration']-datetime.timedelta(days=1)
    RunPeriodSpcialDay_list['Start_Date_Formatted']=RunPeriodSpcialDay_list['Start_Date'].apply(RunPeriodSpecial)
    RunPeriodSpcialDay_list['End_Date_Formatted']=RunPeriodSpcialDay_list['End_Date_Formatted'].map(datetime.datetime.date) 

    return(RunPeriodSpcialDay_list)

RunPeriodSpecialDay_df=RunPeriodSpecialDays(idf1)

## Steps to check whether the start of the disruptive event starts/ends on a special day.This is relevant in constructing schedule for disruptive events

RunPeriodSpecialDay_StartDateCheck_df=RunPeriodSpecialDay_df.loc[((Event_Start_Datetime.date()>=RunPeriodSpecialDay_df['Start_Date_Formatted']) & (Event_Start_Datetime.date()<=RunPeriodSpecialDay_df['End_Date_Formatted']))]
RunPeriodSpecialDay_EndDateCheck_df=RunPeriodSpecialDay_df.loc[((Event_End_Datetime.date()>=RunPeriodSpecialDay_df['Start_Date_Formatted']) & (Event_End_Datetime.date()<=RunPeriodSpecialDay_df['End_Date_Formatted']))]

## Check if start date is on a holiday, if so, list the holiday name
RunPeriodSpecialDay_DateCheck=[0,0]
if RunPeriodSpecialDay_StartDateCheck_df.empty:
    RunPeriodSpecialDay_DateCheck[0] = False
    
elif RunPeriodSpecialDay_StartDateCheck_df.iloc[0]['Special_Day_Type'] == "CustomDay1":
    
    RunPeriodSpecialDay_DateCheck[0] = "CustomDay1"
    
elif RunPeriodSpecialDay_StartDateCheck_df.iloc[0]['Special_Day_Type'] == "CustomDay2":
    
    RunPeriodSpecialDay_DateCheck[0] = "CustomDay2"
    
else: 
    RunPeriodSpecialDay_DateCheck[0] = RunPeriodSpecialDay_StartDateCheck_df.iloc[0]['Name']

## Check if end date is on a holiday, if so, list the holiday name
if RunPeriodSpecialDay_EndDateCheck_df.empty:
    RunPeriodSpecialDay_DateCheck[1] = False
    
elif RunPeriodSpecialDay_EndDateCheck_df.iloc[0]['Special_Day_Type'] == "CustomDay1":
    
    RunPeriodSpecialDay_DateCheck[1] = "CustomDay1"
    
elif RunPeriodSpecialDay_EndDateCheck_df.iloc[0]['Special_Day_Type'] == "CustomDay2":
    
    RunPeriodSpecialDay_DateCheck[1] = "CustomDay2"
    
else: 
    RunPeriodSpecialDay_DateCheck[1] = RunPeriodSpecialDay_EndDateCheck_df.iloc[0]['Name']

EventDate_Analyze_df["Is_Holiday"]=RunPeriodSpecialDay_DateCheck

# Steps to check which day of the week the disruptive event start/end date begins on.

EventDate_Analyze_df["Is_Weekend"]=EventDate_Analyze_df['Is_Weekday'] == False

EventDate_Analyze_df["Is_Monday"]=EventDate_Analyze_df['Event Date'].dt.dayofweek.between(0,0)
EventDate_Analyze_df["Is_Tuesday"]=EventDate_Analyze_df['Event Date'].dt.dayofweek.between(1,1)
EventDate_Analyze_df["Is_Wednesday"]=EventDate_Analyze_df['Event Date'].dt.dayofweek.between(2,2)
EventDate_Analyze_df["Is_Thursday"]=EventDate_Analyze_df['Event Date'].dt.dayofweek.between(3,3)
EventDate_Analyze_df["Is_Friday"]=EventDate_Analyze_df['Event Date'].dt.dayofweek.between(4,4)
EventDate_Analyze_df["Is_Saturday"]=EventDate_Analyze_df['Event Date'].dt.dayofweek.between(5,5)
EventDate_Analyze_df["Is_Sunday"]=EventDate_Analyze_df['Event Date'].dt.dayofweek.between(6,6)


#The resulted analysis dataframe can be used to inform schedule selection
EventDate_Analyze_df

# Defining Functions

In [ ]:
def ListOfSchedules(idf1):

######Purpose of this function is to list all the schedules in a energyplus input file(idf file)##########
######Input of the function is an idf file loaded as an IDF object using the eppy library#################
######Output of this function is a pandas dataframe with all schedule#####################################
    
#List all schedules in idf file
    matching = [s for s in idf1.idfobjects if s.startswith("SCHEDULE:")]

# initialize storage array for schedules in idf file
    idf_schedules=[]

    for match in matching:
        idf_schedules.extend(idf1.idfobjects[match])

    schedule_name=[idf_schedule["Name"] for idf_schedule in idf_schedules]
    schedule_type=[idf_schedule["key"] for idf_schedule in idf_schedules]
    
    schedule_limit_type=[]
    
    for idf_schedule in idf_schedules:
        try:
            schedule_limit_type.append(idf_schedule["Schedule_Type_Limits_Name"])
        except:
            schedule_limit_type.append("")

    schedule_list=pd.DataFrame({'Name':schedule_name,'Type':schedule_type, 'Limit Type':schedule_limit_type})
    
    
    return(schedule_list)


def NewScheduleType_HVAC_Control(idf1):
    
######Purpose of this function is to create a new schedule type - Control#################################
######Input of the function is an idf file loaded as an IDF object using the eppy library#################
######Output of this function is the IDF object with the new schedule type added##########################
    
    #Define Schedule Type - Control - in case it doesn't already exist or named differently
    #-! Need to add logic to check that schedul type does not already exist
    
    newobject = idf1.newidfobject("SCHEDULETypeLimits")
    newobject.Name='Control'
    newobject.Lower_Limit_Value="0"
    newobject.Upper_Limit_Value="4"
    newobject.Numeric_Type="Discrete"

    print(newobject)
    
    return(idf1)

def NewSchedule_Uncontrolled_HVAC(idf1):
    
######Purpose of this function is to create a new schedule - Uncontrolled HVAC - turning off HVAC for full year##########
######Input of the function is an idf file loaded as an IDF object using the eppy library################################
######Output of this function is the IDF object with the new schedule added##############################################
    
    # Create Schedule - Uncontrolled_HVAC 

    newobject = idf1.newidfobject("SCHEDULE:COMPACT")
    newobject.Name='HVAC_Uncontrolled_WholeYear'
    newobject.Schedule_Type_Limits_Name="Control"
    newobject.Field_1="Through:12/31"
    newobject.Field_2="For: AllDays"
    newobject.Field_3="Until: 24:00"
    newobject.Field_4="0"
    
    print("New Schedule added to idf:")
    print(newobject)
    
    return(idf1)


def NewSchedule_PowerOff(idf1):
    
    # Purpose of this function is to create 
    #    a schedule that is always off
    
    # Input of this function is an energyplus input file(idf) 
    #    loaded into an eppy IDF object
    
    # Output of this function(function returns), 
    #    is the eppy IDF object with the added schedule
    
    # A message is also displayed to notify users that a new 
    #    schudle is added to the idf model,and the details of the 
    #    new schedule 
    
    newobject = idf1.newidfobject("SCHEDULE:COMPACT")
    newobject.Name='AllOff_WholeYear'
    newobject.Schedule_Type_Limits_Name="Fraction"
    newobject.Field_1="Through:12/31"
    newobject.Field_2="For: AllDays"
    newobject.Field_3="Until: 24:00"
    newobject.Field_4="0"
    
    print("New Schedule added to idf:")
    print(newobject)
    
    return(idf1)

def save_idf_file(output_filename,idf_output_folder):
    
######Purpose of this function is to save the converted IDF object into a file in a designated folder location ##########
######Input of the function is the output file name and the output folder path ##########################################
######Output of this function is a printed message with the file path ###################################################    

    # Save idf file to specied output folder under specified file name
    
    filepath=idf_output_folder+"\\"+output_filename

    idf1.saveas(filepath)
    
    print(filepath)
    
    return(filepath)

def ListOfFloors(idf1):
    
######Purpose of this function is to analyze the location of the thermal zones and identify which floor they are on#################
######Input of the function is an idf file loaded as an IDF object using the eppy library###########################################
######Output of this function is a count of thermal zones, as well as a list of zones with the floor level they are located in #####    
    
    #List all schedules in idf file
    #Number of zones
    idf_zones=idf1.idfobjects["Zone"]
    idf_zone_len=len(idf_zones)

    #Number of floors

    ##list all floors and zones
    floor_list=[]
    zone_list=[]
    zonelevel_list=[]

    floor_list=[zone["Z_Origin"] for zone in idf_zones]
    zone_list=[zone["Name"] for zone in idf_zones]

    zonelevel_list=pd.DataFrame({'Zone':zone_list,'Z_Origin':floor_list})

    ##find unique z value in zone location
    floor_list=pd.DataFrame(set(floor_list),columns=["Z_Origin"])

    ##order floor levels 
    floor_list_pos=floor_list[floor_list['Z_Origin']>=0].sort_values(by=['Z_Origin']).reset_index(drop=True)
    floor_list_neg=floor_list[floor_list['Z_Origin']<0].sort_values(by=['Z_Origin'],ascending=False).reset_index(drop=True)

    floor_list_pos['floor_tag']=floor_list_pos.index
    floor_list_neg['floor_tag']=(floor_list_neg.index+1)*-1

    floor_list = floor_list_neg.append(floor_list_pos)
    floor_list = floor_list.sort_values(by=['floor_tag'])

    #Add level tag to zone
    zonelevel_list=zonelevel_list.merge(floor_list, how='left', on='Z_Origin')
    
    print("There are {} zones in this model and {} floors".format(idf_zone_len,len(floor_list['floor_tag'].unique())))


    return(zonelevel_list)


def ListOfZoneOccupantSchedule(idf1):
    
######Purpose of this function is to analyze the occupancy of each thermal zone#############################################
######Input of the function is an idf file loaded as an IDF object using the eppy library###################################
######Output of this function is a list of thermal zones with the corresponding occupancy and activity level schedules ##### 
    
    #Occupancy Component
    idf_people=idf1.idfobjects["People"]

    ##list all zones and associated schedules
    schedule_list=[]
    zone_list=[]

    occ_sch_list=[people["Number_of_People_Schedule_Name"] for people in idf_people]
    occ_act_sch_list=[people["Activity_Level_Schedule_Name"] for people in idf_people]
    
    try:
    
        zone_list=[people["Zone_or_ZoneList_Name"] for people in idf_people]
    
    except:
        
        zone_list=[people["Zone_or_ZoneList_or_Space_or_SpaceList_Name"] for people in idf_people]

    zone_occ_list=pd.DataFrame({'Zone':zone_list,'Occupancy Schedule':occ_sch_list, 'Activity Schedule':occ_act_sch_list})


    return(zone_occ_list)
 
    
def NewOutput_Hourly(idf1, variable):
    
######Purpose of this function is to add new hourly output to the simulation model##########
######Input of the function is an idf file loaded as an IDF object using the eppy library###
######Output of this function is the idf object with new hourly output added ##################     

    newobject = idf1.newidfobject("OUTPUT:VARIABLE")
    newobject.Key_Value="*"
    newobject.Variable_Name=variable
    newobject.Reporting_Frequency='Hourly'
    return idf1



def Analysis_Summary (idf_output_df_original,Selected_Scenario):
    
######Purpose of this function is to create summary table to identify zones that require additional analysis################################
######Input of the function is the cleaned energyplus simulation output ####################################################################
######Output of this function is a table with selected thermal zones with the worst overheating/underheating conditions based on db temp ###

    Indicator='Zone Air Temperature [C](Hourly)'

    Results_df=idf_output_df_original
    Results_df_Selected=Results_df[Results_df["FileName"]==Selected_Scenario]
    Results_df_Selected=Results_df_Selected.filter(like=Indicator, axis=1)
    Results_df_Summary=Results_df_Selected.describe().T

    # Analyze Results 


    ## Results summary - Max, Min of Metric based on zone
    Results_df_Summary=Results_df_Summary.reset_index()
    Results_df_Summary=Results_df_Summary.rename(columns={Results_df_Summary.columns[0]: 'Zone:Metric'})
    Results_df_Summary[['Zone','Metric']] = Results_df_Summary['Zone:Metric'].str.split(':',expand=True)
    Results_df_Summary=Results_df_Summary.set_index('Zone:Metric')

    ## Degree Hour -  Heating/Cooling Degree Hour
    upper_temp=31
    lower_temp=15

    Cooling_Critical_Temp=upper_temp
    Heating_Crtitical_Temp=lower_temp

    HDH=lower_temp-Results_df_Selected
    HDH[HDH<0]=0
    Results_df_Summary['HDH']=HDH.sum()

    CDH=Results_df_Selected-upper_temp
    CDH[CDH<0]=0
    Results_df_Summary['CDH']=CDH.sum()

    Results_df_Summary

    #Zone Selection

    Analysis_Zone = pd.DataFrame()
    Analysis_Zone_Num=5


    ## Based on Max Indicator
    Results_df_Summary_Select=Results_df_Summary.sort_values(by='max',ascending=False).reset_index()
    Analysis_Zone['Max_Indicator']=Results_df_Summary_Select['Zone'].head(Analysis_Zone_Num)


    ## Based on Min Indicator
    Results_df_Summary_Select=Results_df_Summary.sort_values(by='min',ascending=False).reset_index()
    Analysis_Zone['Min_Indicator']=Results_df_Summary_Select['Zone'].head(Analysis_Zone_Num)


    ## Based on Max Heating Degree Hour
    Results_df_Summary_Select=Results_df_Summary.sort_values(by='HDH',ascending=False).reset_index()
    Analysis_Zone['HDH']=Results_df_Summary_Select['Zone'].head(Analysis_Zone_Num)

    ## Based on Min Heating Degree Hour
    Results_df_Summary_Select=Results_df_Summary.sort_values(by='CDH',ascending=False).reset_index()
    Analysis_Zone['CDH']=Results_df_Summary_Select['Zone'].head(Analysis_Zone_Num)

    Analysis_Zone=Analysis_Zone.melt()

    Analysis_Zone=Analysis_Zone.rename(columns={"value": "Zone", "variable": "Metric of Interst"})


    Analysis_Zone_Summary=Analysis_Zone.groupby('Zone')['Metric of Interst'].apply(','.join).reset_index()


    Analysis_Zone_Summary=Analysis_Zone_Summary.merge(Results_df_Summary_Select, how='left', on='Zone')

    return Analysis_Zone_Summary


In [ ]:
#Function to read in a compact schedule from idf file and produces a dataframe that extracts fields needed to create datetime object 

def ScheduleConvertCompact(Schedule_Name,Type):
    
######Purpose of this function is to convert compact schedules#################################################################################################
######Input of the function is the schedule name and the type of schedule #####################################################################################
######Output of this function is the schedule object being added to the idf object. As part of the function, the schedule object is added to the idf object ###    

    global Schedule
    global dict_items
    
    
    Sch_Type=Type
    if Sch_Type == "Occupancy":
        Sch_Level="1"
        
    elif Sch_Type == "Equipment":
        Sch_Level="0"
    
    elif Sch_Type == "Zone Control":
        Sch_Level="0"
        
    elif Sch_Type == "Heating_Setpoint":
        Sch_Level="-85"
        
    elif Sch_Type == "Cooling_Setpoint":
        Sch_Level="85"
    
    else:
        print("Schedule Type Not Recognized, level for disruptive event is set to 0.")
        Sch_Level="0"
    
    
    Schedule_Compact=idf1.idfobjects['SCHEDULE:COMPACT']
    
    
    for x in Schedule_Compact:
        if x.Name == Schedule_Name:
        #copies values from schedule to a list 
            dict_items=list(x.values())
    
    #creates data frame from the list with schedule values
    Sch_df=pd.DataFrame(dict_items[0], columns=['config'])
   
    #set schedule name and obtain schedule type
    Sch_Header_Name=Sch_df.iloc[1]['config']
    Sch_Header_Name=Sch_Header_Name+"_TRmod"
    Sch_Header_Type=Sch_df.iloc[0]['config']
    
    #Remove duplicated TRmod schedules
    for x in Schedule_Compact:
        if x.Name == Sch_Header_Name:
            idf1.removeidfobject(x)
    
    #Extracts schedule lines that contains keywords related to schedule construction "Through", "Until", "For" 
    #to label them in corresponding tag columns 
    Sch_df.loc[Sch_df['config'].str.contains("Through", case=False),'Date_Tag']=Sch_df['config']
    Sch_df.loc[Sch_df['config'].str.contains("Until", case=False),'Time_Tag']=Sch_df['config']
    Sch_df.loc[Sch_df['config'].str.contains("For", case=False),'Day_Tag']=Sch_df['config']
    
    #forward fill the tags
    Sch_df.loc[:,'Time_Tag']=Sch_df.loc[:,'Time_Tag'].ffill()
    Sch_df.loc[:,'Date_Tag']=Sch_df.loc[:,'Date_Tag'].ffill()
    Sch_df.loc[:,'Day_Tag']=Sch_df.loc[:,'Day_Tag'].ffill()
    
    #Remove rows that are not needed for the new schedule
    Sch_df=Sch_df[Sch_df['config'].str.contains("For")==False]
    Sch_df=Sch_df[Sch_df['config'].str.contains("Through")==False]
    Sch_df=Sch_df[Sch_df['Date_Tag'].notnull()]
    Sch_df=Sch_df[Sch_df['Time_Tag'].notnull()]

    #Extract elements that are needed to assemble date time object
    Sch_df['Date']=Sch_df['Date_Tag'].str.lower().str.extract(r'through[:]?\s*?(\d{1,2}/\d{1,2})')
    Sch_df['Time']=Sch_df['Time_Tag'].str.lower().str.extract(r'until[:]?\s*?(\d{1,2}:\d{2})')
    Sch_df[['Hour','Minute']]=Sch_df['Time'].str.split(':', expand=True)
    
    #need to replace hour 24 to 0
    Sch_df['Hour_Adjusted']=Sch_df['Hour'].replace('24','0')
    Sch_df[['Month','Day']]=Sch_df['Date'].str.split('/', expand=True).astype(int)
    
    #assemble date time object
    #print(Sch_df)
    
    Sch_df["DateTime"]= pd.to_datetime(str(RunPeriod_Year)+"-"+Sch_df["Month"].astype(str)+"-"+Sch_df["Day"].astype(str)+' '+ Sch_df["Hour_Adjusted"].astype(str)+":"+Sch_df["Minute"].astype(str),format='%Y-%m-%d %H:%M')
    
    #for hour 0, need to add a day as part of hour adjustment
    Sch_df.loc[Sch_df['Hour_Adjusted'] == '0', 'DateTime'] = Sch_df['DateTime']+ datetime.timedelta(days=1)
    
    #extract rows before event start date/time
    Sch_Before_EventStart=Sch_df.loc[Sch_df['DateTime']<Event_Start_Datetime]
    #extract rows after event start date/time
    Sch_After_EventStart=Sch_df.loc[Sch_df['DateTime']>=Event_Start_Datetime]
    #extract rows after event end date/time
    Sch_After_EventEnd=Sch_df.loc[Sch_df['DateTime']>=Event_End_Datetime]
    #extract rows where the through date/time happens between event start and end date
    Sch_Between_Event=Sch_After_EventStart[~Sch_After_EventStart.isin(Sch_After_EventEnd)].dropna()
    
    #For rows that are applicable to before event start date and after event end date, no change is needed
    ##Adding headerlines for those lines
    
    Sch_DayBeforeEventStarts_DateTags=Sch_Before_EventStart['Date_Tag'].unique()

    BeforeEvent_ModConfig_loop=pd.DataFrame([])
    BeforeEvent_df_combined=pd.DataFrame([])

    for datetag in Sch_DayBeforeEventStarts_DateTags:

        BeforeEvent_Line1=datetag
        Sch_Before_EventStart_SelectedDayTag=Sch_Before_EventStart.loc[Sch_Before_EventStart['Date_Tag']==datetag]
        Sch_DayBeforeEventStarts_DayTags=Sch_Before_EventStart_SelectedDayTag['Day_Tag'].unique()
        BeforeEvent_ModConfig_loop_Day=pd.DataFrame([])

        for daytag in Sch_DayBeforeEventStarts_DayTags:

            BeforeEvent_Line2=daytag
            BeforeEvent_df_setup={'config':[BeforeEvent_Line2]}
            BeforeEvent_df=pd.DataFrame(BeforeEvent_df_setup)
            BeforeEvent_ModConfig_DayTagLines=Sch_Before_EventStart_SelectedDayTag.loc[Sch_Before_EventStart_SelectedDayTag['Day_Tag'].str.contains(daytag, case=False,na=False)]
            BeforeEvent_ModConfig_loop_Day=BeforeEvent_ModConfig_loop_Day.append([BeforeEvent_df,BeforeEvent_ModConfig_DayTagLines])
        
        BeforeEvent_df_date_setup={'config':[BeforeEvent_Line1]}
        BeforeEvent_df_date=pd.DataFrame(BeforeEvent_df_date_setup)
        BeforeEvent_ModConfig_loop=BeforeEvent_ModConfig_loop.append([BeforeEvent_df_date,BeforeEvent_ModConfig_loop_Day])
    
    BeforeEvent_df_combined=BeforeEvent_ModConfig_loop
    
    ####After event ends    

    Sch_DayAfterEventEnds_DateTags=Sch_After_EventEnd['Date_Tag'].unique()
    
    AfterEvent_ModConfig_loop=pd.DataFrame([])
    AfterEvent_df_combined=pd.DataFrame([])

    for datetag in Sch_DayAfterEventEnds_DateTags:
        
        AfterEvent_Line1=datetag
        Sch_After_EventEnd_SelectedDayTag=Sch_After_EventEnd.loc[Sch_After_EventEnd['Date_Tag']==datetag]
        Sch_DayAfterEventEnds_DayTags=Sch_After_EventEnd_SelectedDayTag['Day_Tag'].unique()
        AfterEvent_ModConfig_loop_Day=pd.DataFrame([])                                                           
                                                                   
        
        for daytag in Sch_DayAfterEventEnds_DayTags:

            AfterEvent_Line2=daytag
            AfterEvent_df_setup={'config':[AfterEvent_Line2]}
            AfterEvent_df=pd.DataFrame(AfterEvent_df_setup)
            AfterEvent_ModConfig_DayTagLines=Sch_After_EventEnd_SelectedDayTag.loc[Sch_After_EventEnd_SelectedDayTag['Day_Tag'].str.contains(daytag, case=False,na=False)]
            AfterEvent_ModConfig_loop_Day=AfterEvent_ModConfig_loop_Day.append([AfterEvent_df,AfterEvent_ModConfig_DayTagLines])
            
        AfterEvent_df_date_setup={'config':[AfterEvent_Line1]}
        AfterEvent_df_date=pd.DataFrame(AfterEvent_df_date_setup)
        AfterEvent_ModConfig_loop=AfterEvent_ModConfig_loop.append([AfterEvent_df_date,AfterEvent_ModConfig_loop_Day])

    AfterEvent_df_combined=AfterEvent_ModConfig_loop

    
    #Assemble Logic for event
    ## Logic for day before Event Start
    ### Search for rows that belongs to the earliest schedule through date that is after event start
    Sch_DayBefore_Event=Sch_After_EventStart.loc[Sch_After_EventStart['Date']==min(Sch_After_EventStart['Date'])]

    #creating logic to assemble date time(in idf format) for the days before disruptive event starts(where schedule thru date is after event start time)
    Event_DayBeforeStart=Event_Start_Datetime.date()+datetime.timedelta(days=-1)
    Event_DayBeforeStart_idfformat="{}/{:02d}".format(Event_DayBeforeStart.month,Event_DayBeforeStart.day)
    
    Sch_DayBefore_Event_DayTags=Sch_DayBefore_Event['Day_Tag'].unique()
    i=0
    Sch_DayBefore_Event_ModConfig_loop=pd.DataFrame([])
    for daytags in Sch_DayBefore_Event_DayTags:
        
        PreDistruptive_Line2=Sch_DayBefore_Event['Day_Tag'].unique()[i]
        Disruptive_df_DayBefore_setup={'config':[PreDistruptive_Line2]}
        Disruptive_df_DayBefore=pd.DataFrame(Disruptive_df_DayBefore_setup)
        Sch_DayBefore_Event_ModConfig_DayTagLines=Sch_DayBefore_Event[Sch_DayBefore_Event['Day_Tag'].str.contains(daytags)]
        Sch_DayBefore_Event_ModConfig_loop=pd.concat([Sch_DayBefore_Event_ModConfig_loop,Disruptive_df_DayBefore,Sch_DayBefore_Event_ModConfig_DayTagLines])
        i=i+1
        
    PreDistruptive_Line1="Through: {}".format(Event_DayBeforeStart_idfformat)
    Disruptive_df_DayBefore_setup_line1={'config':[PreDistruptive_Line1]}
    Disruptive_df_DayBefore_line1=pd.DataFrame(Disruptive_df_DayBefore_setup_line1)
    
    
    Sch_DayBefore_Event_ModConfig=pd.concat([Disruptive_df_DayBefore_line1,Sch_DayBefore_Event_ModConfig_loop])
    
    ## Logic for Event Start Day
    ###First need to determine which day logic should be used (AllDay, Holiday, {Actual Weekday Days}, Weekend,Weekday or AllOtherDays)
    ### Sch_JustBeforeEvent refers to hours right before event begins
    
    Sch_JustBefore_Event=Sch_DayBefore_Event
    
    if Sch_JustBefore_Event['Day_Tag'].str.contains('AllDays', case=False).any():
    
        Sch_JustBefore_Event_ModConfig_DayTagLines=Sch_DayBefore_Event[Sch_DayBefore_Event['Day_Tag'].str.contains('AllDays', case=False)]

    elif (Sch_JustBefore_Event['Day_Tag'].str.contains('Holidays', case=False).any() and EventDate_Analyze_df["Is_Holiday"].iloc[0] == True):
    
        Sch_JustBefore_Event_ModConfig_DayTagLines=Sch_DayBefore_Event[Sch_DayBefore_Event['Day_Tag'].str.contains('Holidays', case=False)]

    elif (Sch_JustBefore_Event['Day_Tag'].str.contains('Weekend', case=False).any() and EventDate_Analyze_df["Is_Weekend"].iloc[0] == True):
    
        Sch_JustBefore_Event_ModConfig_DayTagLines=Sch_DayBefore_Event[Sch_DayBefore_Event['Day_Tag'].str.contains('Weekend', case=False)] 
    
    elif (Sch_JustBefore_Event['Day_Tag'].str.contains('Weekday', case=False).any() and EventDate_Analyze_df["Is_Weekday"].iloc[0] == True):
    
        Sch_JustBefore_Event_ModConfig_DayTagLines=Sch_DayBefore_Event[Sch_DayBefore_Event['Day_Tag'].str.contains('Weekday', case=False)]
    
    else:
    
        Sch_JustBefore_Event_ModConfig_DayTagLines=Sch_DayBefore_Event[Sch_DayBefore_Event['Day_Tag'].str.contains('AllOtherDays', case=False)]
        
    
    ### Header Line for Event Start Day
    EventStartDay_Line1="Through: {}".format(Event_StartDate)
    EventStartDay_Line2="For: AllDays"
    
    Disruptive_df_EventStartDay_setup={'config':[EventStartDay_Line1,EventStartDay_Line2]}
    Disruptive_df_EventStartDay=pd.DataFrame(Disruptive_df_EventStartDay_setup)

    Disruptive_BeforeEvent=Sch_JustBefore_Event_ModConfig_DayTagLines[Sch_JustBefore_Event_ModConfig_DayTagLines['DateTime'].dt.time<Event_Start_Datetime.time()]
    #remove last 2 rows - until 24:00 - the time for hour 24 will always be the lowest. 
    Disruptive_BeforeEvent=Disruptive_BeforeEvent.drop(Disruptive_BeforeEvent.tail(2).index)
    #add logic for setting until event start time
    Disruptive_BeforeEvent2header="Until: {}".format(Event_StartTime)
    
    Disruptive_BeforeEvent2=Sch_JustBefore_Event_ModConfig_DayTagLines[Sch_JustBefore_Event_ModConfig_DayTagLines['DateTime'].dt.time>=Event_Start_Datetime.time()]
    
    ###TS####
    Disruptive_BeforeEvent2=Sch_JustBefore_Event_ModConfig_DayTagLines[Sch_JustBefore_Event_ModConfig_DayTagLines['DateTime'].dt.time>=Event_Start_Datetime.time()]
    if not Disruptive_BeforeEvent2.size:
        print("list empty")
        Disruptive_BeforeEvent2=Sch_JustBefore_Event_ModConfig_DayTagLines
    else:
        print("list not empty")
    
    ###TS####
    #print(Sch_JustBefore_Event_ModConfig_DayTagLines)
    
    ###TS####



    Disruptive_BeforeEvent2=Disruptive_BeforeEvent2.iloc[1]['config']
    
    
    ### Logic for different end date/time
    Disruptive_TimeDelta = Event_End_Datetime.date() - Event_Start_Datetime.date()
    Disruptive_TimeDelta_Days=Disruptive_TimeDelta.days
    
    #print(Disruptive_TimeDelta_Days)
    
    if Event_End_Datetime.date() == Event_Start_Datetime.date():
        DisruptiveMode = "Same Day"
    elif Disruptive_TimeDelta_Days < 2:
        DisruptiveMode = "Not Same Day"
    else:
        DisruptiveMode = "Not Same Day - Multiple Days"
    
    
    print(DisruptiveMode)
    
    ## if event lasts more than 24 hours
    
    if DisruptiveMode == "Not Same Day - Multiple Days" or DisruptiveMode == "Not Same Day":
        print("mode1")
        EventStartDay_Line3="Until: 24:00"
        EventStartDay_Line4=Sch_Level
        
    elif DisruptiveMode == "Same Day":
        print("mode2")
        EventStartDay_Line3="Until: {}".format(Event_EndTime)
        EventStartDay_Line4=Sch_Level
        
    else:
        print("mode3")
        
    
        
    
    Disruptive_df_EventStartDay_setup_2={'config':[Disruptive_BeforeEvent2header,Disruptive_BeforeEvent2,EventStartDay_Line3,EventStartDay_Line4]}
    
    Disruptive_df_EventStartDay_2=pd.DataFrame(Disruptive_df_EventStartDay_setup_2)

    Disruptive_BeforeEvent_Combined=pd.concat([Disruptive_df_EventStartDay,Disruptive_BeforeEvent,Disruptive_df_EventStartDay_2])
    
    ## Logic for in between event days (day after start and day before end)
    
    Event_DayBeforeEnd=Event_End_Datetime.date()+datetime.timedelta(days=-1)
    Event_DayBeforeEnd_idfformat="{}/{:02d}".format(Event_DayBeforeEnd.month,Event_DayBeforeEnd.day)

    EventInBetweenDay_Line1="Through: {}".format(Event_DayBeforeEnd_idfformat)
    EventInBetweenDay_Line2="For: AllDays" 
    EventInBetweenDay_Line3="Until: 24:00"
    EventInBetweenDay_Line4=Sch_Level

    
    Disruptive_df_EventInBetweenDay_setup={'config':[EventInBetweenDay_Line1,EventInBetweenDay_Line2,EventInBetweenDay_Line3,EventInBetweenDay_Line4]}
    
    Disruptive_df_EventInBetweenDay=pd.DataFrame(Disruptive_df_EventInBetweenDay_setup)
    
    ## Logic for Event End Day

    EventEndDay_Line1="Through: {}".format(Event_EndDate)
    EventEndDay_Line2="For: AllDays"
    EventEndDay_Line3="Until: {}".format(Event_EndTime)
    EventEndDay_Line4=Sch_Level

    Disruptive_df_EventEndDay_setup={'config':[EventEndDay_Line1,EventEndDay_Line2,EventEndDay_Line3,EventEndDay_Line4]}
    Disruptive_df_EventEndDay=pd.DataFrame(Disruptive_df_EventEndDay_setup)
    
    Sch_DayAfter_Event=Sch_After_EventEnd.loc[Sch_After_EventEnd['Date']==min(Sch_After_EventEnd['Date'])]
    Sch_JustAfter_Event=Sch_DayAfter_Event

    if Sch_JustAfter_Event['Day_Tag'].str.contains('AllDays', case=False).any():
    
        Sch_JustAfter_Event_ModConfig_DayTagLines=Sch_DayAfter_Event[Sch_DayAfter_Event['Day_Tag'].str.contains('AllDays', case=False)]

    elif (Sch_JustAfter_Event['Day_Tag'].str.contains('Holidays', case=False).any() and EventDate_Analyze_df["Is_Holiday"].iloc[0] == True):
    
        Sch_JustAfter_Event_ModConfig_DayTagLines=Sch_DayAfter_Event[Sch_DayAfter_Event['Day_Tag'].str.contains('Holidays', case=False)]

    elif (Sch_JustAfter_Event['Day_Tag'].str.contains('Weekend', case=False).any() and EventDate_Analyze_df["Is_Weekend"].iloc[0] == True):
    
        Sch_JustAfter_Event_ModConfig_DayTagLines=Sch_DayAfter_Event[Sch_DayAfter_Event['Day_Tag'].str.contains('Weekend', case=False)] 
    
    elif (Sch_JustAfter_Event['Day_Tag'].str.contains('Weekday', case=False).any() and EventDate_Analyze_df["Is_Weekday"].iloc[0] == True):
    
        Sch_JustAfter_Event_ModConfig_DayTagLines=Sch_DayAfter_Event[Sch_DayAfter_Event['Day_Tag'].str.contains('Weekday', case=False)]
    
    else:
    
        Sch_JustAfter_Event_ModConfig_DayTagLines=Sch_DayAfter_Event[Sch_DayAfter_Event['Day_Tag'].str.contains('AllOtherDays', case=False)]

    
    
    Disruptive_Sch_JustAfter_Event_1=Sch_JustAfter_Event_ModConfig_DayTagLines[Sch_JustAfter_Event_ModConfig_DayTagLines['DateTime'].dt.time>Event_End_Datetime.time()]
    Disruptive_Sch_JustAfter_Event_2=Sch_JustAfter_Event_ModConfig_DayTagLines[Sch_JustAfter_Event_ModConfig_DayTagLines['Hour']=='24']

    Disruptive_Sch_JustAfter_Event=pd.concat([Disruptive_Sch_JustAfter_Event_1,Disruptive_Sch_JustAfter_Event_2])
    
    if DisruptiveMode == "Same Day":
        Disruptive_AfterEvent_Combined=pd.concat([Disruptive_Sch_JustAfter_Event])
        
    else:    
        Disruptive_AfterEvent_Combined=pd.concat([Disruptive_df_EventEndDay,Disruptive_Sch_JustAfter_Event])
        display(HTML(Disruptive_AfterEvent_Combined.to_html()))
    
    
    if DisruptiveMode == "Not Same Day - Multiple Days":
    
        Sch_Mod_Combined=pd.concat([BeforeEvent_df_combined,Sch_DayBefore_Event_ModConfig,Disruptive_BeforeEvent_Combined,Disruptive_df_EventInBetweenDay,Disruptive_AfterEvent_Combined,AfterEvent_df_combined])
        #print(AfterEvent_df_combined)
        display(HTML(AfterEvent_df_combined.to_html()))
    
    elif DisruptiveMode == "Not Same Day":
        Sch_Mod_Combined=pd.concat([BeforeEvent_df_combined,Sch_DayBefore_Event_ModConfig,Disruptive_BeforeEvent_Combined,Disruptive_AfterEvent_Combined,AfterEvent_df_combined])
        
        
    elif DisruptiveMode == "Same Day":
    
        Sch_Mod_Combined=pd.concat([BeforeEvent_df_combined,Sch_DayBefore_Event_ModConfig,Disruptive_BeforeEvent_Combined,Disruptive_AfterEvent_Combined,AfterEvent_df_combined])
    
    Sch_Mod_Combined.reset_index(inplace=True)
    Sch_Mod_Combined["Field_Num"]=Sch_Mod_Combined.index+1
    Sch_Mod_Combined["Field"]="Field_"+Sch_Mod_Combined["Field_Num"].astype(str)
    
    newobject = idf1.newidfobject("Schedule:Compact")
    newobject.Name= Sch_Header_Name
    
    if Sch_Type == "Zone Control":
        newobject.Schedule_Type_Limits_Name="Control Type"
        
    elif (Sch_Type == "Heating_Setpoint") or (Sch_Type == "Cooling_Setpoint"):
        newobject.Schedule_Type_Limits_Name="Temperature"
        
    else:
        newobject.Schedule_Type_Limits_Name="Fraction"
    
    for index, row in Sch_Mod_Combined.iterrows():
        field_name=row['Field']
        newobject[field_name]=row["config"]
    
    
    print("New schedule {} added".format(Sch_Header_Name))
    print(newobject)
    
    return (newobject)


In [ ]:
def ScheduleConvertYearly(Schedule_Name,Type):

######Purpose of this function is to convert yearly schedules#################################################################################################
######Input of the function is the schedule name and the type of schedule #####################################################################################
######Output of this function is the schedule object being added to the idf object. As part of the function, the schedule object is added to the idf object ### 
    
    global Schedule
    global dict_items
    
    Sch_Type=Type
    if Sch_Type == "Occupancy":
        Sch_Level="1"
        
    elif Sch_Type == "Equipment":
        Sch_Level="0"
    
    elif Sch_Type == "Zone Control":
        Sch_Level="0"
        
    elif Sch_Type == "Zone Control":
        Sch_Level="0"
    
    
    else:
        print("Schedule Type Not Recognized, level for disruptive event is set to 0.")
        Sch_Level="0"
    Schedule_Year=idf1.idfobjects['SCHEDULE:YEAR']

    for x in Schedule_Year:
    ##* revised after test 1
        if x.Name == Schedule_Name:
            dict_items=list(x.values())
            dict_name=list(x.objls)

    #creates dataframe from the list with schedule values
    Sch_df=pd.DataFrame(dict_items[0], columns=['config'])
    Sch_df['Key']=dict_name[:len(Sch_df['config'])]

    #set schedule name and obtain schedule type
    Sch_Header_Name=Sch_df.iloc[1]['config']
    Sch_Header_Type=Sch_df.iloc[0]['config']
    Sch_Header_Limit_Type=Sch_df.iloc[2]['config']

    ### Logic for different end date/time
    Disruptive_TimeDelta = Event_End_Datetime.date() - Event_Start_Datetime.date()
    Disruptive_TimeDelta_Days=Disruptive_TimeDelta.days

    if Event_End_Datetime.date() == Event_Start_Datetime.date():
        DisruptiveMode = "Same Day"
    elif Disruptive_TimeDelta_Days < 2:
        DisruptiveMode = "Not Same Day"
    else:
        DisruptiveMode = "Not Same Day - Multiple Days"

    print("DisruptiveMode is {}".format(DisruptiveMode))    

    ## if event lasts more than 24 hours

    if DisruptiveMode == "Not Same Day - Multiple Days" or DisruptiveMode == "Not Same Day":
        EventStartDay_Line3="Until: 24:00"
        EventStartDay_Line4=Sch_Level

    elif DisruptiveMode == "Same Day":
        EventStartDay_Line3="Until: {}".format(Event_EndTime)
        EventStartDay_Line4=Sch_Level


    Sch_df=Sch_df.iloc[3:]
    Sch_df.loc[Sch_df['Key'].str.contains("ScheduleWeek_Name", case=False),'WeekSchName']=Sch_df['config']
    Sch_df.loc[:,'WeekSchName']=Sch_df.loc[:,'WeekSchName'].ffill()
    Sch_df['Key']=Sch_df['Key'].str.rsplit('_', n=1).str[0]
    Sch_df=Sch_df.pivot(index='WeekSchName',columns='Key', values='config').reset_index(drop=True)

    cols=['ScheduleWeek_Name','Start_Month','Start_Day','End_Month','End_Day']

    Sch_df=Sch_df[cols]
    Sch_df['Start_Date']=pd.to_datetime(str(RunPeriod_Year)+"-"+Sch_df["Start_Month"].astype(str)+"-"+Sch_df["Start_Day"].astype(str)+" 00:00",format='%Y-%m-%d %H:%M')
    Sch_df['End_Date']=pd.to_datetime(str(RunPeriod_Year)+"-"+Sch_df["End_Month"].astype(str)+"-"+Sch_df["End_Day"].astype(str)+" 00:00",format='%Y-%m-%d %H:%M')
    Sch_df['End_Date']=Sch_df['End_Date']+ datetime.timedelta(days=1)


    StartSchName=Sch_df.loc[(Sch_df['Start_Date'] <= Event_Start_Datetime) & (Sch_df['End_Date'] > Event_Start_Datetime)]['ScheduleWeek_Name'][0]
    EndSchName=Sch_df.loc[(Sch_df['Start_Date'] <= Event_End_Datetime) & (Sch_df['End_Date'] > Event_End_Datetime)]['ScheduleWeek_Name'][0]

    # Get appropriate weekly schedule

    Week_Sch_Daily_idf=idf1.idfobjects['SCHEDULE:WEEK:DAILY']
    Week_Sch_Compact_idf=idf1.idfobjects['SCHEDULE:WEEK:COMPACT']

    for i in Week_Sch_Daily_idf:    
        if i.Name == StartSchName:
            StartSch=i
            StartSchType="Week:Daily"

        if i.Name == EndSchName:
            EndSch=i
            EndSchType="Week:Daily"

    for i in Week_Sch_Compact_idf:    
        if i.Name == StartSchName:
            StartSch=i
            StartSchType="Week:Compact"

        if i.Name == EndSchName:
            EndSch=i
            EndSchType="Week:Compact"        
    ############################################Find Day Schedule from Week Schedule##################################################        

    if (StartSchType == "Week:Daily" and EndSchType == "Week:Daily"):     
        print('Both start and end weekly schedule type are Week:Daily schedules.')

        # Find Day Schedule for Start Day       
        StartSchWeek_daily_df=pd.DataFrame(list(StartSch.values())[0], columns=['config'])
        StartSchWeek_daily_df['Key']=list(StartSch.values())[1]    
        StartSchWeek=StartSchWeek_daily_df.copy()
        StartSchWeek=StartSchWeek.iloc[2:]

        StartSchWeek['Day']="Is_"+StartSchWeek['Key'].str.split('_', n=1).str[0]

        EventDate_Analyze_Start_Yearly=EventDate_Analyze_df[:1]


        if EventDate_Analyze_Start_Yearly['Is_Holiday'][0] != False:

            if EventDate_Analyze_Start_Yearly['Is_Holiday'][0] == 'CustomDay1':
                StartSchWeek_DaySchName=StartSchWeek.loc[StartSchWeek['Key']=="CustomDay1_ScheduleDay_Name"]
                StartSchWeek_DaySchName=StartSchWeek_DaySchName['config'].iloc[0]

            elif EventDate_Analyze_Start_Yearly['Is_Holiday'][0] == 'CustomDay2':

                StartSchWeek_DaySchName=StartSchWeek.loc[StartSchWeek['Key']=="CustomDay2_ScheduleDay_Name"]
                StartSchWeek_DaySchName=StartSchWeek_DaySchName['config'].iloc[0]

            else:
                StartSchWeek_DaySchName=StartSchWeek.loc[StartSchWeek['Key']=="Holiday_ScheduleDay_Name"]
                StartSchWeek_DaySchName=StartSchWeek_DaySchName['config'].iloc[0]

        else:

            print('not holiday')
            EventDate_Analyze_Start_Yearly=EventDate_Analyze_Start_Yearly.melt()[3:]
            EventDate_Analyze_Start_Yearly=EventDate_Analyze_Start_Yearly.loc[EventDate_Analyze_Start_Yearly['value'] == True]
            EventDate_Analyze_Start_Yearly_Day=EventDate_Analyze_Start_Yearly['variable'].iloc[0]

            StartSchWeek=StartSchWeek.loc[StartSchWeek['Day']==EventDate_Analyze_Start_Yearly_Day]
            StartSchWeek_DaySchName=StartSchWeek['config'].iloc[0]

        # Find Day Schedule for End Day 

        EndSchWeek_df=pd.DataFrame(list(EndSch.values())[0], columns=['config'])

        EndSchWeek_df['Key']=list(EndSch.values())[1]    

        EndSchWeek=EndSchWeek_df.copy()
        EndSchWeek=EndSchWeek[2:]
        EndSchWeek['Day']="Is_"+EndSchWeek['Key'].str.split('_', n=1).str[0]

        EventDate_Analyze_End_Yearly=EventDate_Analyze_df[1:].reset_index(drop=True)

        if EventDate_Analyze_End_Yearly['Is_Holiday'][0] != False:

            if EventDate_Analyze_End_Yearly['Is_Holiday'][0] == 'CustomDay1':
                EndSchWeek_DaySchName=EndSchWeek.loc[EndSchWeek['Key']=="CustomDay1_ScheduleDay_Name"]
                EndSchWeek_DaySchName=EndSchWeek_DaySchName['config'].iloc[0]

            elif EventDate_Analyze_End_Yearly['Is_Holiday'][0] == 'CustomDay2':

                EndSchWeek_DaySchName=EndSchWeek.loc[EndSchWeek['Key']=="CustomDay2_ScheduleDay_Name"]
                EndSchWeek_DaySchName=EndSchWeek_DaySchName['config'].iloc[0]

            else:
                EndSchWeek_DaySchName=EndSchWeek.loc[EndSchWeek['Key']=="Holiday_ScheduleDay_Name"]
                EndSchWeek_DaySchName=EndSchWeek_DaySchName['config'].iloc[0]            
        else:

            EventDate_Analyze_End_Yearly=EventDate_Analyze_End_Yearly.melt()[3:]
            EventDate_Analyze_End_Yearly=EventDate_Analyze_End_Yearly.loc[EventDate_Analyze_End_Yearly['value'] == True]
            EventDate_Analyze_End_Yearly_Day=EventDate_Analyze_End_Yearly['variable'].iloc[0]

            EndSchWeek=EndSchWeek.loc[EndSchWeek['Day']==EventDate_Analyze_End_Yearly_Day]


            EndSchWeek_DaySchName=EndSchWeek['config'].iloc[0] 

    if (StartSchType == "Week:Compact" and EndSchType == "Week:Compact"):

        print("placeholder for week compact")

    ############################################Get and Modify Actual Day Schedule##################################################

    Day_Sch_Hourly_idf=idf1.idfobjects['SCHEDULE:Day:Hourly']
    Day_Sch_Interval_idf=idf1.idfobjects['SCHEDULE:Day:Interval']
    Day_Sch_List_idf=idf1.idfobjects['SCHEDULE:Day:List']

    for i in Day_Sch_Hourly_idf:    
        if i.Name == StartSchWeek_DaySchName:
            StartSchDay=i
            StartSchTypeDay="Day:Hourly"        

        if i.Name == EndSchWeek_DaySchName:
            EndSchDay=i
            EndSchTypeDay="Day:Hourly"

    for i in Day_Sch_Interval_idf:    
        if i.Name == StartSchWeek_DaySchName:
            StartSchDay=i
            StartSchTypeDay="Day:Interval"

        if i.Name == StartSchWeek_DaySchName:
            EndSchDay=i
            EndSchTypeDay="Day:Interval"        

    for i in Day_Sch_List_idf:    
        if i.Name == StartSchWeek_DaySchName:
            StartSchDay=i
            StartSchTypeDay="Day:List"

        if i.Name == StartSchWeek_DaySchName:
            EndSchDay=i
            EndSchTypeDay="Day:List"  

    if (StartSchTypeDay == "Day:Hourly" and EndSchTypeDay == "Day:Hourly"): 

        print('Day:Hourly Schedule is used, but not recommended, since partial hours are not supported. Event will begin at the end on the hour')

        StartSchDay_df=pd.DataFrame(list(StartSchDay.values())[0], columns=['config'])
        StartSchDay_df['Key']=list(StartSchDay.values())[1]
        StartSchDay_Header=StartSchDay_df[:3]
        StartSchDay_Hourly_df=StartSchDay_df[3:].copy()
        StartSchDay_Hourly_df['Hour']=StartSchDay_Hourly_df['Key'].str.rsplit('_', n=1).str[1]

        EndSchDay_df=pd.DataFrame(list(EndSchDay.values())[0], columns=['config'])
        EndSchDay_df['Key']=list(EndSchDay.values())[1]
        EndSchDay_Header=EndSchDay_df[:3]
        EndSchDay_Hourly_df=EndSchDay_df[3:].copy()
        EndSchDay_Hourly_df['Hour']=EndSchDay_Hourly_df['Key'].str.rsplit('_', n=1).str[1]


        Event_Start_Hour=Event_Start_Datetime.hour
        Event_End_Hour=Event_End_Datetime.hour

        if DisruptiveMode == "Same Day":

            print("Disruptive event starts and ends on the same day")

            StartSchDay_Hourly_df.loc[(StartSchDay_Hourly_df['Hour'].astype('int')>=Event_Start_Hour) & (StartSchDay_Hourly_df['Hour'].astype('int')<=Event_End_Hour), ['config']]=Sch_Level
            Day_Sch_df=pd.concat([StartSchDay_Header,StartSchDay_Hourly_df])
            Start_Sch_Header_Name=StartSchDay_Header.loc[1,'config']

            #Remove Existing TRMod schedules if it already exists

            for x in list(Day_Sch_Hourly_idf):
                if x.Name == Start_Sch_Header_Name + "_Day_TRMod":
                    idf1.removeidfobject(x)

            #Add Day Schedule

            newobject = idf1.newidfobject("Schedule:Day:Hourly")
            newobject.Name= Start_Sch_Header_Name + "_Day_TRMod"
            newobject.Schedule_Type_Limits_Name="Fraction"

            for index, row in Day_Sch_df.iterrows():
                field_name=row['Key']
                newobject[field_name]=row["config"]

            ######################################Assemble Week Schedule#####################################

            #Remove Existing TRMod schedules if it already exists 
            for x in list(Week_Sch_Compact_idf):
                if x.Name == StartSchName + 'TRWeekSch_DisruptDay':
                    print("")
                    idf1.removeidfobject(x)

            #create new week schedule
            newobject = idf1.newidfobject("Schedule:Week:Compact")
            newobject['Name']=StartSchName + 'TRWeekSch_DisruptDay'
            newobject['DayType_List_1']='For AllDays'
            newobject['ScheduleDay_Name_1']= Start_Sch_Header_Name + "_Day_TRMod"


        elif DisruptiveMode == "Not Same Day - Multiple Days" or DisruptiveMode == "Not Same Day":

            print("Disruptive event starts and ends on separate days")

            StartSchDay_Hourly_df.loc[StartSchDay_Hourly_df['Hour'].astype('int')>=Event_Start_Hour, ['config']]=Sch_Level
            EndSchDay_Hourly_df.loc[EndSchDay_Hourly_df['Hour'].astype('int')< Event_End_Hour, ['config']]=Sch_Level
            InBetween_Hourly_df=StartSchDay_Hourly_df.copy()
            InBetween_Hourly_df['config']=Sch_Level       

            #Remove Existing TRMod schedules if it already exists 
            for x in list(Day_Sch_Hourly_idf):
                if (x.Name == Start_Sch_Header_Name + "_StartDay_TRMod") or (x.Name == End_Sch_Header_Name + "_EndDay_TRMod") or (x.Name == Sch_Type + "_InBetweenSch_TRMod"):
                    idf1.removeidfobject(x)

            #Add Start Day Schedule
            newobject = idf1.newidfobject("Schedule:Day:Hourly")
            Start_Sch_Header_Name=StartSchDay_Header.loc[1,'config']
            newobject.Name= Start_Sch_Header_Name + "_StartDay_TRMod"
            newobject.Schedule_Type_Limits_Name="Fraction"

            for index, row in StartSchDay_Hourly_df.iterrows():
                field_name=row['Key']
                newobject[field_name]=row["config"]

            #Add End Day Schedule

            newobject = idf1.newidfobject("Schedule:Day:Hourly")
            End_Sch_Header_Name=EndSchDay_Header.loc[1,'config']
            newobject.Name= End_Sch_Header_Name + "_EndDay_TRMod"
            newobject.Schedule_Type_Limits_Name="Fraction"

            for index, row in EndSchDay_Hourly_df.iterrows():
                field_name=row['Key']
                newobject[field_name]=row["config"]

            #Add In-between Day Schedule

            newobject = idf1.newidfobject("Schedule:Day:Hourly")
            newobject.Name= Sch_Type + "_InBetweenSch_TRMod"
            newobject.Schedule_Type_Limits_Name="Fraction"

            for index, row in InBetween_Hourly_df.iterrows():
                field_name=row['Key']
                newobject[field_name]=row["config"]


            #######################################Assemble Week Schedules ##########################################    
            #Remove Existing TRMod schedules if it already exists

            for x in list(Week_Sch_Compact_idf):
                if (x.Name == StartSchName + 'TRWeekSch_Start') or (x.Name == EndSchName + 'TRWeekSch_End') or (x.Name == Sch_Type + 'TRWeekSch_InBetween'):
                    idf1.removeidfobject(x)

            newobject = idf1.newidfobject("Schedule:Week:Compact")
            newobject['Name']=StartSchName + 'TRWeekSch_Start'
            newobject['DayType_List_1']='For AllDays'
            newobject['ScheduleDay_Name_1']=Start_Sch_Header_Name + "_StartDay_TRMod"

            newobject = idf1.newidfobject("Schedule:Week:Compact")
            newobject['Name']=EndSchName +'TRWeekSch_End'
            newobject['DayType_List_1']='For AllDays'
            newobject['ScheduleDay_Name_1']=End_Sch_Header_Name + "_EndDay_TRMod"

            newobject = idf1.newidfobject("Schedule:Week:Compact")
            newobject['Name']=Sch_Type + 'TRWeekSch_InBetween'
            newobject['DayType_List_1']='For AllDays'
            newobject['ScheduleDay_Name_1']=Sch_Type + "_InBetweenSch_TRMod"


    if (StartSchTypeDay == "Day:Interval" and EndSchTypeDay == "Day:Interval"):

        print('Day:Interval Schedule is used')

        StartSchDay_df=pd.DataFrame(list(StartSchDay.values())[0], columns=['config'])
        StartSchDay_Header=StartSchDay_df[:4]
        StartSchDay_Hourly_df=StartSchDay_df[4:].copy()
        
        print(StartSchDay_Hourly_df)
        
        ##* Modified after test 1 
        
        if StartSchDay_Hourly_df['config'].str.contains("Until", case=False,na=False).sum() >= 1:
        
            StartSchDay_Hourly_df.loc[StartSchDay_Hourly_df['config'].str.contains("Until", case=False,na=False),'Time_Tag']=StartSchDay_Hourly_df['config']
        else:
            StartSchDay_Hourly_df.loc[StartSchDay_Hourly_df['config'].str.contains(":", case=False,na=False),'Time_Tag']="Until: "+StartSchDay_Hourly_df['config'].astype("string")
            StartSchDay_Hourly_df.loc[StartSchDay_Hourly_df['config'].str.contains(":", case=False,na=False),'config']="Until: "+StartSchDay_Hourly_df['config'].astype("string")
        
        StartSchDay_Hourly_df.loc[:,'Time_Tag']=StartSchDay_Hourly_df.loc[:,'Time_Tag'].ffill()
        StartSchDay_Hourly_df['Time_Tag']=StartSchDay_Hourly_df['Time_Tag'].str.strip()
        StartSchDay_Hourly_df=StartSchDay_Hourly_df[StartSchDay_Hourly_df['config'].str.contains("Until",case=False,na=False)==False]
        StartSchDay_Hourly_df['Time']=StartSchDay_Hourly_df['Time_Tag'].str.split(':',1).str[-1]
        StartSchDay_Hourly_df[['Hour','Minute']]=StartSchDay_Hourly_df['Time'].str.split(':', expand=True) 
        
        StartSchDay_Hourly_df['Hour']=StartSchDay_Hourly_df['Hour'].str.strip()
        StartSchDay_Hourly_df['Minute']=StartSchDay_Hourly_df['Minute'].str.strip()
    
        StartSchDay_Hourly_df['Hour_Adjusted']=StartSchDay_Hourly_df['Hour'].str.replace('24','0')
        StartSchDay_Hourly_df['Date']=Event_Start_Datetime.date()
        StartSchDay_Hourly_df['Date']=StartSchDay_Hourly_df['Date'].astype(str)
        StartSchDay_Hourly_df["DateTime"]= pd.to_datetime(StartSchDay_Hourly_df['Date'].astype(str)+' '+ StartSchDay_Hourly_df["Hour_Adjusted"].astype(str)+":"+StartSchDay_Hourly_df["Minute"].astype(str),format='%Y-%m-%d %H:%M')
        StartSchDay_Hourly_df.loc[StartSchDay_Hourly_df['Hour_Adjusted'] == '0', 'DateTime'] = StartSchDay_Hourly_df['DateTime']+ datetime.timedelta(days=1)
        Start_Sch_Header_Name=StartSchDay_Header.loc[1,'config']

        #extract rows before event start date/time
        Sch_Before_EventStart=StartSchDay_Hourly_df.loc[StartSchDay_Hourly_df['DateTime']<Event_Start_Datetime]
        Sch_Before_EventStart=Sch_Before_EventStart.reset_index(drop=True)
        #extract rows after event start date/time
        Sch_After_EventStart=StartSchDay_Hourly_df.loc[StartSchDay_Hourly_df['DateTime']>=Event_Start_Datetime]
        Sch_After_EventStart=Sch_After_EventStart.reset_index(drop=True)

        Disruptive_BeforeEventStart_Sch_Level=Sch_After_EventStart['config'].loc[0]
        Disruptive_StartDayBeforeEventStart=pd.DataFrame(columns=['config','Time_Tag'])
        if Event_StartTime != "00:00":
            Disruptive_StartDayBeforeEventStart.loc[0]=[Disruptive_BeforeEventStart_Sch_Level,"Until: {}".format(Event_StartTime)]

        EndSchDay_df=pd.DataFrame(list(EndSchDay.values())[0], columns=['config'])
        EndSchDay_Header=EndSchDay_df[:4]
        EndSchDay_Hourly_df=EndSchDay_df[4:].copy()
        
        if EndSchDay_Hourly_df['config'].str.contains("Until", case=False,na=False).sum() >= 1:
        
            EndSchDay_Hourly_df.loc[EndSchDay_Hourly_df['config'].str.contains("Until", case=False,na=False),'Time_Tag']=EndSchDay_Hourly_df['config']
        else:
            EndSchDay_Hourly_df.loc[EndSchDay_Hourly_df['config'].str.contains(":", case=False,na=False),'config']="Until: "+EndSchDay_Hourly_df['config'].astype("string")
        
        
        EndSchDay_Hourly_df.loc[EndSchDay_Hourly_df['config'].str.contains("Until", case=False,na=False),'Time_Tag']=EndSchDay_Hourly_df['config']
        EndSchDay_Hourly_df.loc[:,'Time_Tag']=EndSchDay_Hourly_df.loc[:,'Time_Tag'].ffill()
        EndSchDay_Hourly_df['Time_Tag']=EndSchDay_Hourly_df['Time_Tag'].str.strip()
        EndSchDay_Hourly_df=EndSchDay_Hourly_df[EndSchDay_Hourly_df['config'].str.contains("Until",case=False,na=False)==False]
        EndSchDay_Hourly_df['Time']=EndSchDay_Hourly_df['Time_Tag'].str.split(':',1).str[-1]
        EndSchDay_Hourly_df[['Hour','Minute']]=EndSchDay_Hourly_df['Time'].str.split(':', expand=True) 
        
        EndSchDay_Hourly_df['Hour']=EndSchDay_Hourly_df['Hour'].str.strip()
        EndSchDay_Hourly_df['Minute']=EndSchDay_Hourly_df['Minute'].str.strip()
    
        EndSchDay_Hourly_df['Hour_Adjusted']=EndSchDay_Hourly_df['Hour'].str.replace('24','0')
        EndSchDay_Hourly_df['Date']=Event_End_Datetime.date()
        EndSchDay_Hourly_df['Date']=EndSchDay_Hourly_df['Date'].astype(str)
        EndSchDay_Hourly_df["DateTime"]= pd.to_datetime(EndSchDay_Hourly_df['Date'].astype(str)+' '+ EndSchDay_Hourly_df["Hour_Adjusted"].astype(str)+":"+EndSchDay_Hourly_df["Minute"].astype(str),format='%Y-%m-%d %H:%M')
        EndSchDay_Hourly_df.loc[EndSchDay_Hourly_df['Hour_Adjusted'] == '0', 'DateTime'] = EndSchDay_Hourly_df['DateTime']+ datetime.timedelta(days=1)
        End_Sch_Header_Name=EndSchDay_Header.loc[1,'config']


        #extract rows after event end date/time
        Sch_After_EventEnd=EndSchDay_Hourly_df.loc[EndSchDay_Hourly_df['DateTime']>=Event_End_Datetime]


        #Assemble Day Schedule for different disruptive modes
        #Event start and end on same day
        if DisruptiveMode == "Same Day":

            Disruptive_EventSameDay=pd.DataFrame(columns=['config','Time_Tag'])
            Disruptive_EventSameDay.loc[0]=[Sch_Level,"Until: {}".format(Event_EndTime)]

            Day_Sch_df=pd.concat([Sch_Before_EventStart,Disruptive_StartDayBeforeEventStart,Disruptive_EventSameDay,Sch_After_EventEnd])
            Day_Sch_df=Day_Sch_df.rename(columns={"config":"Value_Until_Time"})
            Day_Sch_df=Day_Sch_df.drop(columns=['Time','Hour','Minute','Hour_Adjusted','Date','DateTime']).reset_index(drop=True)
            Day_Sch_df=Day_Sch_df[["Time_Tag","Value_Until_Time"]]
            Day_Sch_df=Day_Sch_df.reset_index()
            Day_Sch_df=Day_Sch_df.melt(id_vars=['index']).reset_index().sort_values(by=['index','level_0'])
            Day_Sch_df["index"]=Day_Sch_df["index"]+1
            Day_Sch_df.loc[Day_Sch_df['variable'].str.contains('Value_Until_Time', case=False),'variable']=Day_Sch_df['variable']+"_"+Day_Sch_df['index'].astype('string')
            Day_Sch_df.loc[Day_Sch_df['variable'].str.contains('Time_Tag', case=False),'variable']="Time_"+Day_Sch_df['index'].astype('string')
            Day_Sch_df=Day_Sch_df.rename(columns={"variable":"Key","value":"config"})

            #Remove Existing TRMod schedules if it already exists

            for x in list(Day_Sch_Interval_idf):
                if x.Name == Start_Sch_Header_Name + "_Day_TRMod":
                    idf1.removeidfobject(x)

            #Add Day Schedule

            newobject = idf1.newidfobject("SCHEDULE:Day:Interval")
            newobject.Name= Start_Sch_Header_Name + "_Day_TRMod"
            newobject.Schedule_Type_Limits_Name="Fraction"

            for index, row in Day_Sch_df.iterrows():
                field_name=row['Key']
                newobject[field_name]=row["config"]

            ######################################Assemble Week Schedule#####################################

            #Remove Existing TRMod schedules if it already exists 
            for x in list(Week_Sch_Compact_idf):
                if x.Name == StartSchName + 'TRWeekSch_DisruptDay':
                    print("Item {} removed".format(x.Name))
                    idf1.removeidfobject(x)

            #create new week schedule
            newobject = idf1.newidfobject("Schedule:Week:Compact")
            newobject['Name']=StartSchName + 'TRWeekSch_DisruptDay'
            newobject['DayType_List_1']='For AllDays'
            newobject['ScheduleDay_Name_1']= Start_Sch_Header_Name + "_Day_TRMod"


        if DisruptiveMode == "Not Same Day - Multiple Days" or DisruptiveMode == "Not Same Day":    
            print("The event starts and ends on separate days")

            Disruptive_EventStartDay=pd.DataFrame(columns=['config','Time_Tag'])
            Disruptive_EventStartDay.loc[0]=[Sch_Level,"Until: 24:00"]

            StartDay_Sch_df=pd.concat([Sch_Before_EventStart,Disruptive_StartDayBeforeEventStart,Disruptive_EventStartDay])

            StartDay_Sch_df=StartDay_Sch_df.rename(columns={"config":"Value_Until_Time"})
            StartDay_Sch_df=StartDay_Sch_df.drop(columns=['Time','Hour','Minute','Hour_Adjusted','Date','DateTime']).reset_index(drop=True)
            StartDay_Sch_df=StartDay_Sch_df[["Time_Tag","Value_Until_Time"]]
            StartDay_Sch_df=StartDay_Sch_df.reset_index()
            StartDay_Sch_df=StartDay_Sch_df.melt(id_vars=['index']).reset_index().sort_values(by=['index','level_0'])
            StartDay_Sch_df["index"]=StartDay_Sch_df["index"]+1
            StartDay_Sch_df.loc[StartDay_Sch_df['variable'].str.contains('Value_Until_Time', case=False),'variable']=StartDay_Sch_df['variable']+"_"+StartDay_Sch_df['index'].astype('string')
            StartDay_Sch_df.loc[StartDay_Sch_df['variable'].str.contains('Time_Tag', case=False),'variable']="Time_"+StartDay_Sch_df['index'].astype('string')
            StartDay_Sch_df=StartDay_Sch_df.rename(columns={"variable":"Key","value":"config"})

            Disruptive_EventEndDay=pd.DataFrame(columns=['config','Time_Tag'])
            Disruptive_EventEndDay.loc[0]=[Sch_Level,"Until: {}".format(Event_EndTime)]

            EndDay_Sch_df=pd.concat([Disruptive_EventEndDay,Sch_After_EventEnd])

            EndDay_Sch_df=EndDay_Sch_df.rename(columns={"config":"Value_Until_Time"})
            EndDay_Sch_df=EndDay_Sch_df.drop(columns=['Time','Hour','Minute','Hour_Adjusted','Date','DateTime']).reset_index(drop=True)
            EndDay_Sch_df=EndDay_Sch_df[["Time_Tag","Value_Until_Time"]]
            EndDay_Sch_df=EndDay_Sch_df.reset_index()
            EndDay_Sch_df=EndDay_Sch_df.melt(id_vars=['index']).reset_index().sort_values(by=['index','level_0'])
            EndDay_Sch_df["index"]=EndDay_Sch_df["index"]+1
            EndDay_Sch_df.loc[EndDay_Sch_df['variable'].str.contains('Value_Until_Time', case=False),'variable']=EndDay_Sch_df['variable']+"_"+EndDay_Sch_df['index'].astype('string')
            EndDay_Sch_df.loc[EndDay_Sch_df['variable'].str.contains('Time_Tag', case=False),'variable']="Time_"+EndDay_Sch_df['index'].astype('string')
            EndDay_Sch_df=EndDay_Sch_df.rename(columns={"variable":"Key","value":"config"})

            Disruptive_EventBetweenDay=pd.DataFrame(columns=['config','Time_Tag'])
            Disruptive_EventBetweenDay.loc[0]=[Sch_Level,"Until: 24:00"]

            BetweenDay_Sch_df=Disruptive_EventBetweenDay

            BetweenDay_Sch_df=BetweenDay_Sch_df.rename(columns={"config":"Value_Until_Time"})
            #BetweenDay_Sch_df=BetweenDay_Sch_df.drop(columns=['Time','Hour','Minute','Hour_Adjusted','Date','DateTime']).reset_index(drop=True)
            BetweenDay_Sch_df=BetweenDay_Sch_df[["Time_Tag","Value_Until_Time"]]
            BetweenDay_Sch_df=BetweenDay_Sch_df.reset_index()
            BetweenDay_Sch_df=BetweenDay_Sch_df.melt(id_vars=['index']).reset_index().sort_values(by=['index','level_0'])
            BetweenDay_Sch_df["index"]=BetweenDay_Sch_df["index"]+1
            BetweenDay_Sch_df.loc[BetweenDay_Sch_df['variable'].str.contains('Value_Until_Time', case=False),'variable']=BetweenDay_Sch_df['variable']+"_"+BetweenDay_Sch_df['index'].astype('string')
            BetweenDay_Sch_df.loc[BetweenDay_Sch_df['variable'].str.contains('Time_Tag', case=False),'variable']="Time_"+BetweenDay_Sch_df['index'].astype('string')
            BetweenDay_Sch_df=BetweenDay_Sch_df.rename(columns={"variable":"Key","value":"config"})

            for x in list(Day_Sch_Interval_idf):
                if (x.Name == Start_Sch_Header_Name + "_StartDay_TRMod") or (x.Name == End_Sch_Header_Name + "_EndDay_TRMod") or (x.Name == Sch_Type + "_InBetweenSch_TRMod"):
                    print("Item {} removed".format(x.Name))
                    idf1.removeidfobject(x)

            #Add Start Day Schedule

            newobject = idf1.newidfobject("SCHEDULE:Day:Interval")
            newobject.Name= Start_Sch_Header_Name + "_StartDay_TRMod"
            newobject.Schedule_Type_Limits_Name="Fraction"

            for index, row in StartDay_Sch_df.iterrows():
                field_name=row['Key']
                newobject[field_name]=row["config"]               


            #Add End Day Schedule

            newobject = idf1.newidfobject("SCHEDULE:Day:Interval")
            newobject.Name= End_Sch_Header_Name + "_EndDay_TRMod"
            newobject.Schedule_Type_Limits_Name="Fraction"

            for index, row in EndDay_Sch_df.iterrows():
                field_name=row['Key']
                newobject[field_name]=row["config"]

            #Add In-between Day Schedule

            newobject = idf1.newidfobject("SCHEDULE:Day:Interval")
            newobject.Name= Sch_Type + "_InBetweenSch_TRMod"
            newobject.Schedule_Type_Limits_Name="Fraction"

            for index, row in BetweenDay_Sch_df.iterrows():
                field_name=row['Key']
                newobject[field_name]=row["config"]  


            #######################################Assemble Week Schedules ##########################################    
            #Remove Existing TRMod schedules if it already exists

            for x in list(Week_Sch_Compact_idf):
                if (x.Name == StartSchName + 'TRWeekSch_Start') or (x.Name == EndSchName + 'TRWeekSch_End') or (x.Name == Sch_Type + 'TRWeekSch_InBetween'):
                    print("Item {} removed".format(x.Name))
                    idf1.removeidfobject(x)


            newobject = idf1.newidfobject("Schedule:Week:Compact")
            newobject['Name']=StartSchName + 'TRWeekSch_Start'
            newobject['DayType_List_1']='For AllDays'
            newobject['ScheduleDay_Name_1']=Start_Sch_Header_Name + "_StartDay_TRMod"

            newobject = idf1.newidfobject("Schedule:Week:Compact")
            newobject['Name']=EndSchName +'TRWeekSch_End'
            newobject['DayType_List_1']='For AllDays'
            newobject['ScheduleDay_Name_1']=End_Sch_Header_Name + "_EndDay_TRMod"

            newobject = idf1.newidfobject("Schedule:Week:Compact")
            newobject['Name']=Sch_Type + 'TRWeekSch_InBetween'
            newobject['DayType_List_1']='For AllDays'
            newobject['ScheduleDay_Name_1']=Sch_Type + "_InBetweenSch_TRMod"



    ###########################################Assemble Year Schedule############################################################


    #extract rows before event start date/time - schedules that begin and end before event starts
    Sch_Before_EventStart=Sch_df.loc[Sch_df['End_Date']<Event_Start_Datetime]

    #extract rows after event start date/time - schedules that starts before event starts
    Sch_After_EventStart=Sch_df.loc[(Sch_df['Start_Date']<=Event_Start_Datetime) & (Sch_df['End_Date']>=Event_Start_Datetime)]

    Sch_After_EventStart_Mod=Sch_After_EventStart.copy()
    Sch_After_EventStart_Mod['End_Date']=Event_Start_Datetime - datetime.timedelta(days=1)
    Sch_After_EventStart_Mod['End_Month']=Sch_After_EventStart_Mod['End_Date'][0].month
    Sch_After_EventStart_Mod['End_Day']=Sch_After_EventStart_Mod['End_Date'][0].day

    #extract rows starts before event end date/time and ends after event end date/time
    Sch_Between_EventEnd=Sch_df.loc[(Sch_df['Start_Date']<=Event_End_Datetime) & (Sch_df['End_Date']>=Event_End_Datetime)]

    Sch_Between_EventEnd_Mod=Sch_Between_EventEnd.copy()
    Sch_Between_EventEnd_Mod['Start_Date']=Event_End_Datetime + datetime.timedelta(days=1)
    Sch_Between_EventEnd_Mod['Start_Month']=Sch_Between_EventEnd_Mod['Start_Date'][0].month
    Sch_Between_EventEnd_Mod['Start_Day']=Sch_Between_EventEnd_Mod['Start_Date'][0].day

    #extract rows after event end date/time
    Sch_After_EventEnd=Sch_df.loc[(Sch_df['Start_Date']>Event_End_Datetime) & (Sch_df['End_Date']>Event_End_Datetime)]


    #Assemble Distruptive Event

    if DisruptiveMode == "Same Day":

        Sch_Distruptive=Sch_After_EventStart_Mod.copy()
        Sch_Distruptive['ScheduleWeek_Name']=StartSchName + 'TRWeekSch_DisruptDay'
        Sch_Distruptive['Start_Date']=Event_Start_Datetime
        Sch_Distruptive['End_Date']=Event_Start_Datetime
        Sch_Distruptive['Start_Month']=Sch_Distruptive['Start_Date'][0].month
        Sch_Distruptive['Start_Day']=Sch_Distruptive['Start_Date'][0].day
        Sch_Distruptive['End_Month']=Sch_Distruptive['End_Date'][0].month
        Sch_Distruptive['End_Day']=Sch_Distruptive['End_Date'][0].day

        Sch_TR=pd.concat([Sch_After_EventStart_Mod,Sch_Distruptive,Sch_Between_EventEnd_Mod,Sch_After_EventEnd])


    elif DisruptiveMode == "Not Same Day":
        print("1")
        Sch_Distruptive_Start=Sch_After_EventStart_Mod.copy()
        Sch_Distruptive_Start['ScheduleWeek_Name']=StartSchName + 'TRWeekSch_Start'
        Sch_Distruptive_Start['Start_Date']=Event_Start_Datetime
        Sch_Distruptive_Start['End_Date']=Event_Start_Datetime
        Sch_Distruptive_Start['Start_Month']=Sch_Distruptive_Start['Start_Date'][0].month
        Sch_Distruptive_Start['Start_Day']=Sch_Distruptive_Start['Start_Date'][0].day
        Sch_Distruptive_Start['End_Month']=Sch_Distruptive_Start['End_Date'][0].month
        Sch_Distruptive_Start['End_Day']=Sch_Distruptive_Start['End_Date'][0].day

        Sch_Distruptive_End=Sch_After_EventStart_Mod.copy()
        Sch_Distruptive_End['ScheduleWeek_Name']=EndSchName +'TRWeekSch_End'
        Sch_Distruptive_End['Start_Date']=Event_End_Datetime
        Sch_Distruptive_End['End_Date']=Event_End_Datetime
        Sch_Distruptive_End['Start_Month']=Sch_Distruptive_End['Start_Date'][0].month
        Sch_Distruptive_End['Start_Day']=Sch_Distruptive_End['Start_Date'][0].day
        Sch_Distruptive_End['End_Month']=Sch_Distruptive_End['End_Date'][0].month
        Sch_Distruptive_End['End_Day']=Sch_Distruptive_End['End_Date'][0].day

        Sch_TR=pd.concat([Sch_After_EventStart_Mod,Sch_Distruptive_Start,Sch_Distruptive_End,Sch_Between_EventEnd_Mod,Sch_After_EventEnd])


    elif DisruptiveMode == "Not Same Day - Multiple Days":
        Sch_Distruptive_Start=Sch_After_EventStart_Mod.copy()
        Sch_Distruptive_Start['ScheduleWeek_Name']=StartSchName + 'TRWeekSch_Start'
        Sch_Distruptive_Start['Start_Date']=Event_Start_Datetime
        Sch_Distruptive_Start['End_Date']=Event_Start_Datetime
        Sch_Distruptive_Start['Start_Month']=Sch_Distruptive_Start['Start_Date'][0].month
        Sch_Distruptive_Start['Start_Day']=Sch_Distruptive_Start['Start_Date'][0].day
        Sch_Distruptive_Start['End_Month']=Sch_Distruptive_Start['End_Date'][0].month
        Sch_Distruptive_Start['End_Day']=Sch_Distruptive_Start['End_Date'][0].day

        Sch_Distruptive_InBetween=Sch_After_EventStart_Mod.copy()
        Sch_Distruptive_InBetween['ScheduleWeek_Name']=Sch_Type + 'TRWeekSch_InBetween'
        Sch_Distruptive_InBetween['Start_Date']=Event_Start_Datetime + datetime.timedelta(days=1)
        Sch_Distruptive_InBetween['End_Date']=Event_End_Datetime + datetime.timedelta(days=-1)
        Sch_Distruptive_InBetween['Start_Month']=Sch_Distruptive_InBetween['Start_Date'][0].month
        Sch_Distruptive_InBetween['Start_Day']=Sch_Distruptive_InBetween['Start_Date'][0].day
        Sch_Distruptive_InBetween['End_Month']=Sch_Distruptive_InBetween['End_Date'][0].month
        Sch_Distruptive_InBetween['End_Day']=Sch_Distruptive_InBetween['End_Date'][0].day

        Sch_Distruptive_End=Sch_After_EventStart_Mod.copy()
        Sch_Distruptive_End['ScheduleWeek_Name']=EndSchName +'TRWeekSch_End'
        Sch_Distruptive_End['Start_Date']=Event_End_Datetime
        Sch_Distruptive_End['End_Date']=Event_End_Datetime
        Sch_Distruptive_End['Start_Month']=Sch_Distruptive_End['Start_Date'][0].month
        Sch_Distruptive_End['Start_Day']=Sch_Distruptive_End['Start_Date'][0].day
        Sch_Distruptive_End['End_Month']=Sch_Distruptive_End['End_Date'][0].month
        Sch_Distruptive_End['End_Day']=Sch_Distruptive_End['End_Date'][0].day

        Sch_TR=pd.concat([Sch_After_EventStart_Mod,Sch_Distruptive_Start,Sch_Distruptive_InBetween,Sch_Distruptive_End,Sch_Between_EventEnd_Mod,Sch_After_EventEnd])


    Sch_TR=Sch_TR.reset_index(drop=True) 
    Sch_TR=Sch_TR.reset_index()
    Sch_TR['index']=Sch_TR['index']+1

    Sch_TR=Sch_TR.drop(columns=['Start_Date','End_Date'])
    Sch_TR=Sch_TR.melt(id_vars=['index'])
    Sch_TR['Key']=Sch_TR['Key']+"_"+Sch_TR['index'].astype(str)
    Sch_TR=Sch_TR.reset_index()

    Sch_TR=Sch_TR.sort_values(by=['index', 'level_0'])
    Sch_TR=Sch_TR.drop(columns=['level_0','index'])

    #Remove Existing TRMod schedules if it already exists
    for x in list(Schedule_Year):
        if x.Name == Sch_Header_Name+"_TRMod":
            idf1.removeidfobject(x)


    newobject = idf1.newidfobject("Schedule:Year")
    newobject['Name']=Sch_Header_Name+"_TRMod"
    if Sch_Type == "Zone Control":
        newobject.Schedule_Type_Limits_Name="Control Type"
        
    elif (Sch_Type == "Heating_Setpoint") or (Sch_Type == "Cooling_Setpoint"):
        newobject.Schedule_Type_Limits_Name="Temperature"
        
    else:
        newobject.Schedule_Type_Limits_Name="Fraction"
    
    
    for index, row in Sch_TR.iterrows():
        field_name=row['Key']
        newobject[field_name]=row["value"]

    print("new schedule added"+Sch_Header_Name+"_TRMod")
    
    return(newobject)

In [ ]:
def ScheduleConvertConstant(Schedule_Name,Type):

######Purpose of this function is to convert constant schedules#################################################################################################
######Input of the function is the schedule name and the type of schedule #####################################################################################
######Output of this function is the schedule object being added to the idf object. As part of the function, the schedule object is added to the idf object ###
    
    
    global Schedule
    global dict_items

    Sch_Type=Type
    if Sch_Type == "Occupancy":
        Sch_Level="1"

    elif Sch_Type == "Equipment":
        Sch_Level="0"

    elif Sch_Type == "Zone Control":
        Sch_Level="0"

    elif Sch_Type == "Zone Control":
        Sch_Level="0"

    ### Logic for different end date/time
    Disruptive_TimeDelta = Event_End_Datetime.date() - Event_Start_Datetime.date()
    Disruptive_TimeDelta_Days=Disruptive_TimeDelta.days    

    if Event_End_Datetime.date() == Event_Start_Datetime.date():
        DisruptiveMode = "Same Day"
    elif Disruptive_TimeDelta_Days < 2:
        DisruptiveMode = "Not Same Day"
    else:
        DisruptiveMode = "Not Same Day - Multiple Days"    

    Schedule_Constant=idf1.idfobjects['SCHEDULE:CONSTANT']    

    Event_DayBeforeStart=Event_Start_Datetime.date() + datetime.timedelta(days=-1)
    Event_DayBeforeEnd=Event_End_Datetime.date() + datetime.timedelta(days=-1)

    Event_DayBeforeStart_idfformat="{}/{:02d}".format(Event_DayBeforeStart.month,Event_DayBeforeStart.day)
    Event_DayBeforeEnd_idfformat="{}/{:02d}".format(Event_DayBeforeEnd.month,Event_DayBeforeEnd.day)


    for x in Schedule_Constant:
     ##* revised after test 1
         if x.Name == Schedule_Name:
             ScheduelType_Limits_Name=x['Schedule_Type_Limits_Name']
             ScheduelType_hourly_Level=x['Hourly_Value']

    ##logic for the day before                
    DayBeforeEventSch=pd.DataFrame(columns=['config'])
    DayBeforeEventSch.loc[0]=["Through: {}".format(Event_DayBeforeStart_idfformat)]
    DayBeforeEventSch.loc[1]=["For: AllDays"]
    DayBeforeEventSch.loc[2]=["Until: 24:00"]
    DayBeforeEventSch.loc[3]=[ScheduelType_hourly_Level]

    ##logic for the start day
    EventStartDaySch=pd.DataFrame(columns=['config'])
    EventStartDaySch.loc[0]=["Through: {}".format(Event_StartDate)]
    EventStartDaySch.loc[1]=["For: AllDays"]
    EventStartDaySch.loc[2]=["Until: {}".format(Event_StartTime)]
    EventStartDaySch.loc[3]=[ScheduelType_hourly_Level]

    if not Event_StartTime == "24:00":
        EventStartDaySch.loc[4]=["Until: 24:00"]
        EventStartDaySch.loc[5]=[Sch_Level]

    ##logic for day before end       
    EventInBetweenDaySch=pd.DataFrame(columns=['config'])
    EventInBetweenDaySch.loc[0]=["Through: {}".format(Event_DayBeforeEnd_idfformat)]
    EventInBetweenDaySch.loc[1]=["For: AllDays"]
    EventInBetweenDaySch.loc[2]=["Until: 24:00"]
    EventInBetweenDaySch.loc[3]=[Sch_Level]


    EndEventDaySch=pd.DataFrame(columns=['config'])
    EndEventDaySch.loc[0]=["Through: {}".format(Event_EndDate)]
    EndEventDaySch.loc[1]=["For: AllDays"]
    EndEventDaySch.loc[2]=["Until: {}".format(Event_EndTime)]
    EndEventDaySch.loc[3]=[Sch_Level]    
    if not Event_EndTime == "24:00":
        EndEventDaySch.loc[4]=["Until: 24:00"]
        EndEventDaySch.loc[5]=[ScheduelType_hourly_Level]

    AfterEndEventDaySch=pd.DataFrame(columns=['config'])
    AfterEndEventDaySch.loc[0]=["Through: 12/31"]
    AfterEndEventDaySch.loc[1]=["For: AllDays"]
    AfterEndEventDaySch.loc[2]=["Until: 24:00"]
    AfterEndEventDaySch.loc[3]=[ScheduelType_hourly_Level]  

    if DisruptiveMode == "Same Day":
        EventSameDaySch=pd.DataFrame(columns=['config'])
        EventSameDaySch.loc[0]=["Through: {}".format(Event_StartDate)]
        EventSameDaySch.loc[1]=["For: AllDays"]
        EventSameDaySch.loc[2]=["Until: {}".format(Event_StartTime)]
        EventSameDaySch.loc[3]=[ScheduelType_hourly_Level]
        EventSameDaySch.loc[4]=["Until: {}".format(Event_EndTime)]
        EventSameDaySch.loc[5]=[Sch_Level]
        if not Event_EndTime == "24:00":
            EventSameDaySch.loc[6]=["Until: 24:00"]
            EventSameDaySch.loc[7]=[ScheduelType_hourly_Level]


        Sch_df=pd.concat([DayBeforeEventSch,EventSameDaySch,AfterEndEventDaySch])

    if DisruptiveMode == "Not Same Day":

        Sch_df=pd.concat([DayBeforeEventSch,EventStartDaySch,EndEventDaySch,AfterEndEventDaySch])

    if DisruptiveMode == "Not Same Day - Multiple Days":

        Sch_df=pd.concat([DayBeforeEventSch,EventStartDaySch,EventInBetweenDaySch,EndEventDaySch,AfterEndEventDaySch])

    Sch_df=Sch_df.reset_index(drop=True).reset_index()
    Sch_df['index']="Field_"+(Sch_df['index']+1).astype('string')


    Schedule_Compact=idf1.idfobjects['SCHEDULE:COMPACT']    
    #Remove Existing TRMod schedules if it already exists
    for x in list(Schedule_Compact):
        if x.Name == Schedule_Name +"_TRMod":
            idf1.removeidfobject(x)

    newobject = idf1.newidfobject("SCHEDULE:COMPACT")
    newobject['Name']=Schedule_Name +"_TRMod"
    newobject['Schedule_Type_Limits_Name']= ScheduelType_Limits_Name

    for index, row in Sch_df.iterrows():
        field_name=row['index']
        newobject[field_name]=row["config"]

    print("new schedule added"+Schedule_Name+"_TRMod")

    return(newobject)    


In [ ]:
def ScheduleConvert(Schedule_Name,Schedule_Function_Type):

######Purpose of this function is to execute schedule conversion by selecting the correct schedule conversion function##############################################################
######Input of the function is the schedule name and the type of schedule ##########################################################################################################
######Output of this function is the schedule object being added to the idf object. As part of the conversion function execution, the schedule object is added to the idf object ###

    Schedule_Name=Schedule_Name
    Schedule_Function_Type=Schedule_Function_Type
    Sch_List=ListOfSchedules(idf1)

    Schedule_Type=Sch_List.loc[Sch_List['Name']==Schedule_Name,'Type'].iloc[0]

    if Schedule_Type.upper() == "SCHEDULE:YEAR":
        newobject=ScheduleConvertYearly(Schedule_Name,Schedule_Function_Type)

    elif Schedule_Type.upper() == "SCHEDULE:COMPACT":
        newobject=ScheduleConvertCompact(Schedule_Name,Schedule_Function_Type)
        
    elif Schedule_Type.upper() == "SCHEDULE:CONSTANT":
        newobject=ScheduleConvertConstant(Schedule_Name,Schedule_Function_Type)
    
    return(newobject)


# Convert and Apply Modified Schedules to Occupancy and Equipment 

In [ ]:
#create modified occupancy schedules
ZoneList=ListOfZoneOccupantSchedule(idf1)
ZoneList_OccSch=ZoneList['Occupancy Schedule'].unique()
ZoneList_OccSch

for sch in ZoneList_OccSch:
    ScheduleConvert(sch,"Occupancy")
    
#Assign Occupancy Schedules 
OccupancyObjects=idf1.idfobjects['People']
for object in OccupancyObjects:
    if "_TRmod" not in object.Number_of_People_Schedule_Name:
        object.Number_of_People_Schedule_Name=object.Number_of_People_Schedule_Name+"_TRmod"
    

In [ ]:
#The following steps are to apply schedule conversion to different equipment, additional equipment can be applied to the apporpriate list.

#Creating a list of Schedules

Sch_List=ListOfSchedules(idf1)
Sch_List['Name_Lower']=Sch_List['Name'].str.lower()

##########################################  Electric Equipment  #############################################   
Equipment_List=[
'ElectricEquipment',
'Lights',
'Exterior:Lights',
'GasEquipment',
'OtherEquipment',
'Exterior:FuelEquipment'
]

Equipment_Sch_Name_List=[]

for equipment in Equipment_List:
    try:
        equip_idf=idf1.idfobjects[equipment]    
        for e in equip_idf: 
            Equipment_Sch_Name_List.append(e.Schedule_Name)

            if (not e.Schedule_Name.strip()=='') and (not e.Schedule_Name.endswith("TRmod")): 
                e.Schedule_Name=e.Schedule_Name+"_TRmod"
    except:
        continue       

Equipment_Sch_Name_List=list(set(Equipment_Sch_Name_List))

Equipment_Sch_List=pd.DataFrame({'Schedule Name':Equipment_Sch_Name_List})

Equipment_Sch_List=Equipment_Sch_List.merge(Sch_List,left_on='Schedule Name', right_on='Name').drop(columns='Name')


for equip_ind in Equipment_Sch_List.index:
       
    Sch_name=Equipment_Sch_List['Schedule Name'][equip_ind]
    ScheduleConvert(Sch_name,"Equipment") 

####################################################  HVAC  #################################################        

#HVAC Equipment
HVAC_Equipment_List=[
'ZoneHVAC:EnergyRecoveryVentilator',
'AirTerminal:SingleDuct:Uncontrolled',
'Fan:OnOff',
'AirLoopHVAC:Unitary:Furnace:HeatCool',
'Airloophvac:Unitaryheatpump:Watertoair',
'Fan:VariableVolume',
'Fan:ConstantVolume',
'ElectricLoadCenter:Transformer'
]

HVAC_Equipment_Sch_Name_List=[]

for equipment in HVAC_Equipment_List:

    ##* added after first test
    try:
    
        equip_idf=idf1.idfobjects[equipment]    
        for e in equip_idf: 
            
            if (not e.Availability_Schedule_Name.strip()=='') and (not e.Availability_Schedule_Name.endswith("TRmod")):
                HVAC_Equipment_Sch_Name_List.append(e.Availability_Schedule_Name)
                e.Availability_Schedule_Name=e.Availability_Schedule_Name+"_TRmod"
                #print(e.Availability_Schedule_Name)
        
    except:
        
        continue
        
HVAC_Equipment_Sch_Name_List=list(set(HVAC_Equipment_Sch_Name_List))

HVAC_Equipment_Sch_List=pd.DataFrame({'Schedule Name':HVAC_Equipment_Sch_Name_List})
HVAC_Equipment_Sch_List['Schedule Name Lower']=HVAC_Equipment_Sch_List['Schedule Name'].str.lower()

HVAC_Equipment_Sch_List=HVAC_Equipment_Sch_List.merge(Sch_List,left_on='Schedule Name Lower', right_on='Name_Lower')

for equip_ind in HVAC_Equipment_Sch_List.index:
    
    Sch_name=HVAC_Equipment_Sch_List['Name'][equip_ind]
    
    #print(Sch_name)
    ScheduleConvert(Sch_name,"Equipment") 

#Simple Ventilation
vent=idf1.idfobjects['ZoneVentilation:DesignFlowRate']      

vent_name_list=[]
for v in vent:
    vent_name_list.append(v.Schedule_Name)
    if not v.Schedule_Name.endswith("TRmod"):
        v.Schedule_Name=v.Schedule_Name+"_TRmod"
    
Vent_Sch_List=pd.DataFrame({'Schedule Name':vent_name_list})

Vent_Sch_List=Vent_Sch_List.merge(Sch_List,left_on='Schedule Name', right_on='Name').drop(columns='Name')

for vent_ind in Vent_Sch_List.index:
    
    Sch_name=Vent_Sch_List['Schedule Name'][vent_ind]
    #print(Sch_name)
    ScheduleConvert(Sch_name,"Equipment") 

#HVAC Control

HVAC_Control=idf1.idfobjects['ZoneControl:Thermostat']      

HVAC_Control_name_list=[]
for h in HVAC_Control:
    HVAC_Control_name_list.append(h.Control_Type_Schedule_Name)
    if not h.Control_Type_Schedule_Name.endswith("TRmod"):
        h.Control_Type_Schedule_Name=h.Control_Type_Schedule_Name+"_TRmod"
    
HVAC_Control_List=pd.DataFrame({'Schedule Name':HVAC_Control_name_list})

HVAC_Control_List=HVAC_Control_List.merge(Sch_List,left_on='Schedule Name', right_on='Name').drop(columns='Name')

for hvac_ind in HVAC_Control_List.index:
    
    Sch_name=HVAC_Control_List['Schedule Name'][hvac_ind]
    ScheduleConvert(Sch_name,"Zone Control") 

####################################################  WaterUse  #################################################       
    
#WaterUse
water=idf1.idfobjects['Wateruse:Equipment']      

water_name_list=[]
for w in water:
    water_name_list.append(w.Flow_Rate_Fraction_Schedule_Name)
    if not w.Flow_Rate_Fraction_Schedule_Name.endswith("TRmod"):
        w.Flow_Rate_Fraction_Schedule_Name=w.Flow_Rate_Fraction_Schedule_Name+"_TRmod"
    
Water_Sch_List=pd.DataFrame({'Schedule Name':water_name_list})

Water_Sch_List=Water_Sch_List.merge(Sch_List,left_on='Schedule Name', right_on='Name').drop(columns='Name')

for water_ind in Water_Sch_List.index:
    
    Sch_name=Water_Sch_List['Schedule Name'][water_ind]
    #print(Sch_name)
    ScheduleConvert(Sch_name,"Equipment")        

In [ ]:
#Display schedule list
sch_list=ListOfSchedules(idf1)
sch_list
display(HTML(sch_list.to_html()))

# Set Output Variables

In [ ]:
OutputVariables=["Zone Air Temperature",
                 "Site Outdoor Air Drybulb Temperature",
                 "Zone People Occupant Count", 
                 "Zone Air Heat Balance System Air Transfer Rate",
                 "Zone Ventilation Fan Electric Energy",
                 "Zone Air Heat Balance System Convective Heat Gain Rate",
                 "Zone Mechanical Ventilation Air Changes per Hour",
                 "Facility Total Building Electric Demand Power",
                 "Facility Total HVAC Electric Demand Power",
                 "Facility Total Electric Demand Power",
                 "Facility Total Building Electricity Demand Rate",
                 "Facility Total HVAC Electricity Demand Rate",
                 "Facility Total Electricity Demand Rate",
                 "Lights Electric Power",
                 "Lights Electricity Rate"
                ]



for v in OutputVariables:
    variable=v
    OutputVariable=idf1.idfobjects["OUTPUT:VARIABLE"]

    matching = [v for v in idf1.idfobjects["OUTPUT:VARIABLE"] if v.Variable_Name == variable]

    if not OutputVariable:
        print("Output variable list was empty, new Output Variable {} added".format(variable))
        newobject = idf1.newidfobject("OUTPUT:VARIABLE")
        newobject.Key_Value="*"
        newobject.Variable_Name=variable
        newobject.Reporting_Frequency='Hourly'
        
    else:
    
        if not matching:
            print("New Output Variable {} added".format(variable))
            newobject = idf1.newidfobject("OUTPUT:VARIABLE")
            newobject.Key_Value="*"
            newobject.Variable_Name=variable
            newobject.Reporting_Frequency='Hourly'
            
        else:
        
             for obj in OutputVariable:
                if obj.Variable_Name == variable:
                    print("Varible {} already in model, modified specification for thermal resilience analysis".format(variable))
                    obj.Key_Value="*"
                    obj.Reporting_Frequency='Hourly'
                
                        


# Remove Day Light Saving Time Objects

In [ ]:
# Daylight saving time logic creates error at the midnight time step on the day disruptive event occurs

idf_DaylightSavings=idf1.idfobjects['RUNPERIODCONTROL:DAYLIGHTSAVINGTIME']


idf_DaylightSavings_len=len(idf_DaylightSavings)
i=0
while i < idf_DaylightSavings_len:
    idf_DaylightSavings.pop(i)
    i=i+1


# Save converted file

In [ ]:
filepath=save_idf_file(output_filename,idf_output_folder)

# Analysis

In [ ]:
# Read output file

RunPeriod_Year=2023

# the folder path should be a folder with csv files that contains hourly energy plus simulation outputs
# multiple hourly output csv files can be in the folder for different simualtion scenarios, the script joins the results in one data table
FolderPath=r"C:\Users\Desktop\Output"
FolderPath_Files = glob.glob(os.path.join(FolderPath , "*.csv"))

ListofFiles=[]
idf_output_df=[]
df=[]

for files in FolderPath_Files:
    df=pd.read_csv(files, index_col=None, header=0)
    df.insert(loc=1, column='FileName',value=os.path.basename(files))
    ListofFiles.append(df)

idf_output_df=pd.concat(ListofFiles,axis=0, ignore_index=True)
idf_output_df_original=idf_output_df

idf_output_df.insert(loc=1, column='Time', value=idf_output_df['Date/Time'].str.split(' ').str[-1])
idf_output_df.insert(loc=2, column='Hour', value=idf_output_df['Time'].str.split(':').str[0])
idf_output_df.insert(loc=3, column='Hour_Adjusted', value=idf_output_df['Hour'].replace('24','0'))
idf_output_df.insert(loc=4, column='Minute', value=idf_output_df['Time'].str.split(':').str[1])
idf_output_df.insert(loc=5, column='Date', value=idf_output_df['Date/Time'].str.split(' ').str[1])
idf_output_df.insert(loc=6, column='Month', value=idf_output_df['Date'].str.split('/').str[0])
idf_output_df.insert(loc=7, column='Day', value=idf_output_df['Date'].str.split('/').str[-1])

idf_output_df.insert(loc=8, column='DateTime', value=pd.to_datetime(str(RunPeriod_Year)+"-"+idf_output_df['Month']+"-"+idf_output_df['Day']+' '+ idf_output_df["Hour_Adjusted"]+":"+idf_output_df["Minute"],format='%Y-%m-%d %H:%M'))

idf_output_df.loc[idf_output_df['Hour_Adjusted'] == '0', 'DateTime'] = idf_output_df['DateTime']+ datetime.timedelta(days=1)

idf_output_df=idf_output_df.melt(id_vars=['FileName','Date/Time','Time','Hour','Hour_Adjusted','Minute','Date','Month','Day','DateTime'])

idf_output_df.insert(loc=10, column='Zone', value=idf_output_df['variable'].str.split(':').str[0])
idf_output_df.insert(loc=11, column='Metric', value=idf_output_df['variable'].str.split(':').str[-1])

idf_output_df['Metric']=idf_output_df['Metric'].str.strip()

# Analytics Dashboard

In [ ]:
#Analytics Dashboard
app=dash.Dash()
Scenario_Selection_List=idf_output_df['FileName'].unique()
Zone_Selection_List=idf_output_df['Zone'].unique()
Metric_Selection_List=idf_output_df['Metric'].unique()


app.layout=dcc.Tabs([
    
            #Thermal Resilience Analytics Dashboard
            dcc.Tab(label='Thermal Resilience Analytics Dashboard', children=[
            html.Div([html.H1('Thermal Resilience Analytics Dashboard'),
            
            html.Label("Select Simulation Scenario"),
            dcc.Dropdown(id='Simulation_Scenario',options=Scenario_Selection_List),
            
            html.Label("Select Zone"),
            dcc.Dropdown(id='Zone_Selection',options=Zone_Selection_List),
            dcc.Tabs([
            
            dcc.Tab(label='Simulation Results Summary', children=[
            html.Div([
            html.Label("Enter Upper Threshold  "),
            dcc.Input(id='UpperThreshold_summary',value=25,type='number'),
            html.Label("Enter Lower Threshold  "),
            dcc.Input(id='LowerThreshold_summary',value=18,type='number')]),            
                
                html.Div(
            dash_table.DataTable(id='filtered_summary',data=[],columns=[]))    
                     
            ]),    
                
            dcc.Tab(label='Metric-Time Graph', children=[
            html.Div([
            html.Label("Select Metric"),
            dcc.Dropdown(id='Metric_Selection',options=Metric_Selection_List)]),
            html.Div([
            html.Label("Enter Upper Threshold  "),
            dcc.Input(id='UpperThreshold',value=25,type='number'),
            html.Label("Enter Lower Threshold  "),
            dcc.Input(id='LowerThreshold',value=18,type='number')]),
            html.Div(dcc.Graph(id='temp_graph'))          
            ]),
            
            dcc.Tab(label='Carpet Plot', children=[    
            html.Div([
            html.Div([
            html.Label("Enter Upper Threshold  "),
            dcc.Input(id='UpperThreshold_DegHr',value=25,type='number'),
            html.Label("Enter Lower Threshold  "),
            dcc.Input(id='LowerThreshold_DegHr',value=18,type='number')]),       
            ]),
            dcc.Graph(id='carpet_plot'),
            html.Div([
            dash_table.DataTable(id='filtered_TA_summary',data=[],columns=[])
            ])
               
            ])
            
            ])    
                
            ])
            ])
            ])
            
                     
                
        
@app.callback(Output('temp_graph','figure'),
             [Input('Zone_Selection','value'),
             Input('Metric_Selection','value'),
             Input('Simulation_Scenario','value'),
             Input('UpperThreshold','value'),
             Input('LowerThreshold','value')
             ])

def update_figure(selected_zone, selected_metric,selected_scenario, upper_temp, lower_temp):
    
    filtered_idf_output_df1=idf_output_df[(idf_output_df['Zone']==selected_zone)&(idf_output_df['FileName']==selected_scenario) & (idf_output_df['Metric']==selected_metric)]
    fig=plty.graph_objects.Figure()
    fig.add_trace(plty.graph_objects.Scatter(x=filtered_idf_output_df1.DateTime,y=filtered_idf_output_df1.value))
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text='{} [\u00b0 C]'.format(selected_metric))
    fig.add_hrect(y0=lower_temp, y1=upper_temp, line_width=0, fillcolor="green", opacity=0.2,annotation_text="Upper Limit", annotation_position="inside top right")
    fig.add_hrect(y0=lower_temp, y1=upper_temp, line_width=0, fillcolor="green", opacity=0,annotation_text="Lower Limit", annotation_position="inside bottom right")
    fig.add_vrect(x0=Event_Start_Datetime, x1=Event_End_Datetime, line_width=0, fillcolor="grey", opacity=0.2,annotation_text="Distruptive Event", annotation_position="inside top left")
    fig.update_layout(showlegend=False,title={'text':"Time-{} Plot".format(selected_metric), 'y':0.9,
        'x':0.5, 'xanchor': 'center',
        'yanchor': 'top'})

    fig.update_layout(font=dict(size=16))

    return fig

@app.callback(Output('carpet_plot','figure'),
             [Input('Zone_Selection','value'),Input('Simulation_Scenario','value'),Input('UpperThreshold_DegHr','value'),Input('LowerThreshold_DegHr','value')
              
             ])

def update_figure2(selected_zone1,selected_scenario1,upper_temp,lower_temp):

    
    filtered_idf_output_df_eval=idf_output_df[(idf_output_df['Zone']==selected_zone1) & (idf_output_df['FileName']==selected_scenario1)& (idf_output_df['Metric']=='Zone Air Temperature [C](Hourly)')]
    filtered_idf_output_df_eval["Day"]=filtered_idf_output_df_eval["DateTime"].dt.date
    filtered_idf_output_df_eval["Hour"]=filtered_idf_output_df_eval["DateTime"].dt.time
    
    
    filtered_idf_output_df_eval.loc[filtered_idf_output_df_eval['value'].between(lower_temp,upper_temp),'condition'] ='Normal'
    filtered_idf_output_df_eval.loc[filtered_idf_output_df_eval['value']<lower_temp,'condition'] ='Too Cold'
    filtered_idf_output_df_eval.loc[filtered_idf_output_df_eval['value']>upper_temp,'condition'] ='Too Hot'

    filtered_idf_output_df_eval.loc[filtered_idf_output_df_eval['condition']=='Normal','condition_num'] = 0.5
    filtered_idf_output_df_eval.loc[filtered_idf_output_df_eval['condition']=='Too Cold','condition_num'] = 0
    filtered_idf_output_df_eval.loc[filtered_idf_output_df_eval['condition']=='Too Hot','condition_num'] = 1
    
    
    filtered_idf_output_df_eval_matrix=filtered_idf_output_df_eval.pivot(index=['Hour'],columns='Day',values='condition_num')
    color_scheme=[(0, '#636efa'),  (0.33, '#636efa'),(0.33, '#9ACD32'), (0.66, '#9ACD32'),(0.66, '#ef553b'),  (1.00, '#ef553b')]
    fig2=px.imshow(filtered_idf_output_df_eval_matrix,color_continuous_scale=color_scheme)

    layout = px.imshow(filtered_idf_output_df_eval_matrix,color_continuous_scale=color_scheme).layout

    fig2.layout.coloraxis = layout.coloraxis
    fig2.update_layout(coloraxis_colorbar=dict(
        title="Conditions",
        tickvals=[0.1,0.5,0.9],
        ticktext=["Too Cold","Normal","Too Hot"],
        lenmode="pixels", len=300))
    fig2.update_coloraxes(cmax=1, cmin=0)
    fig2.update_layout(title={'text':"Zone Air Temperature Carpet Plot", 'y':0.95,
        'x':0.5, 'xanchor': 'center',
        'yanchor': 'top'})
    
    fig2.update_layout(font=dict(size=16))
    
    
    return fig2

@app.callback(Output('filtered_summary','data'),Output('filtered_summary','columns'),
             [Input('Simulation_Scenario','value'),Input('UpperThreshold_summary','value'),Input('LowerThreshold_summary','value')
             ])

def filteredTable(selected_scenario2,upper_temp,lower_temp):
    
    Indicator='Zone Air Temperature [C](Hourly)'
    Results_df=idf_output_df_original
    Results_df_Selected=Results_df[Results_df["FileName"]==selected_scenario2]
    Results_df_Selected=Results_df_Selected.filter(like=Indicator, axis=1)
    Results_df_Summary=Results_df_Selected.describe().T

    # Analyze Results 


    ## Results summary - Max, Min of Metric based on zone
    Results_df_Summary=Results_df_Summary.reset_index()
    Results_df_Summary=Results_df_Summary.rename(columns={Results_df_Summary.columns[0]: 'Zone:Metric'})
    Results_df_Summary[['Zone','Metric']] = Results_df_Summary['Zone:Metric'].str.split(':',expand=True)
    Results_df_Summary=Results_df_Summary.set_index('Zone:Metric')

    Cooling_Critical_Temp=upper_temp
    Heating_Crtitical_Temp=lower_temp

    HDH=lower_temp-Results_df_Selected
    HDH[HDH<0]=0
    Results_df_Summary['Underheating Degree Hours']=round(HDH.sum(),1)

    CDH=Results_df_Selected-upper_temp
    CDH[CDH<0]=0
    Results_df_Summary['Overheating Degree Hours']=round(CDH.sum(),1)

    Results_df_Summary

    #Zone Selection

    Analysis_Zone = pd.DataFrame()
    Analysis_Zone_Num=5


    ## Based on Max Indicator
    Results_df_Summary_Select=Results_df_Summary.sort_values(by='max',ascending=False).reset_index()
    Results_df_Summary['max']=round(Results_df_Summary['max'],1)
    Analysis_Zone['Max_Indicator']=Results_df_Summary_Select['Zone'].head(Analysis_Zone_Num)


    ## Based on Min Indicator
    Results_df_Summary_Select=Results_df_Summary.sort_values(by='min',ascending=False).reset_index()
    Results_df_Summary['min']=round(Results_df_Summary['min'],1)
    Analysis_Zone['Min_Indicator']=Results_df_Summary_Select['Zone'].head(Analysis_Zone_Num)


    ## Based on Max Heating Degree Hour
    Results_df_Summary_Select=Results_df_Summary.sort_values(by='Underheating Degree Hours',ascending=False).reset_index()
    Analysis_Zone['Underheating Degree Hours']=Results_df_Summary_Select['Zone'].head(Analysis_Zone_Num)

    ## Based on Min Heating Degree Hour
    Results_df_Summary_Select=Results_df_Summary.sort_values(by='Overheating Degree Hours',ascending=False).reset_index()
    Analysis_Zone['Overheating Degree Hours']=Results_df_Summary_Select['Zone'].head(Analysis_Zone_Num)

    Analysis_Zone=Analysis_Zone.melt()

    Analysis_Zone=Analysis_Zone.rename(columns={"value": "Zone", "variable": "Metric of Interst"})


    Analysis_Zone_Summary=Analysis_Zone.groupby('Zone')['Metric of Interst'].apply(','.join).reset_index()


    Analysis_Zone_Summary=Analysis_Zone_Summary.merge(Results_df_Summary_Select, how='left', on='Zone')
    Analysis_Zone_Summary1=Analysis_Zone_Summary.drop(columns=['mean','std','25%','50%','75%','Zone:Metric'], axis=1)

    return Analysis_Zone_Summary1.to_dict('records'),[{"name": i, "id": i} for i in Analysis_Zone_Summary1.columns]

@app.callback(Output('filtered_TA_summary','data'),Output('filtered_TA_summary','columns'),
             [Input('Zone_Selection','value'),
             Input('Simulation_Scenario','value'),
             Input('UpperThreshold_DegHr','value'),
             Input('LowerThreshold_DegHr','value')
             ])


def update_TA_summary_table(selected_zone2,selected_scenario2,upper_temp,lower_temp):
    
    
    filtered_idf_output_df_eval_TA=idf_output_df[(idf_output_df['Zone']==selected_zone2) & (idf_output_df['FileName']==selected_scenario2)& (idf_output_df['Metric']=='Zone Air Temperature [C](Hourly)')]
    filtered_idf_output_df_eval_TA["Day"]=filtered_idf_output_df_eval_TA["DateTime"].dt.date
    filtered_idf_output_df_eval_TA["Hour"]=filtered_idf_output_df_eval_TA["DateTime"].dt.time
    
   
    filtered_idf_output_df_eval_TA.loc[filtered_idf_output_df_eval_TA['value'].between(lower_temp,upper_temp),'condition'] ='Normal'
    filtered_idf_output_df_eval_TA.loc[filtered_idf_output_df_eval_TA['value']<lower_temp,'condition'] ='Too Cold'
    filtered_idf_output_df_eval_TA.loc[filtered_idf_output_df_eval_TA['value']>upper_temp,'condition'] ='Too Hot'
    
    Analysis_Zone_Summary_TA=pd.DataFrame()
    
    Analysis_Zone_Summary_TA.at[0,'Proportion of Year Too Hot']=round(len(filtered_idf_output_df_eval_TA[filtered_idf_output_df_eval_TA['condition']=="Too Hot"])/8760,2)
    Analysis_Zone_Summary_TA.at[0,'Proportion of Year Too Cold']=round(len(filtered_idf_output_df_eval_TA[filtered_idf_output_df_eval_TA['condition']=="Too Cold"])/8760,2)
    Analysis_Zone_Summary_TA.at[0,'Proportion of Year Normal']=round(len(filtered_idf_output_df_eval_TA[filtered_idf_output_df_eval_TA['condition']=="Normal"])/8760,2)

    
    return Analysis_Zone_Summary_TA.to_dict('records'),[{"name": i, "id": i} for i in Analysis_Zone_Summary_TA.columns]

###################################################################################################


                             
if __name__=='__main__':
    app.run_server()
    